# <span style='color:blue'> SMALL SLOPE APPROXIMATION IN LAYERED MEDIA </span>

## Código basado en Phys. Rev. B (63) 245411 ##

### https://journals.aps.org/prb/abstract/10.1103/PhysRevB.63.245411 ###
### https://arxiv.org/pdf/cond-mat/0009329.pdf ###

### PRB: ###
- rough suface separating two layered media; 
- rough surface on the upper side;
- rough surface on the bottom side.

### Two random rough surfaces: basado en  J. Opt. Soc. Am. A 18, 2778-2788 (2001) ###


### https://www.osapublishing.org/josaa/abstract.cfm?uri=JOSAA-18-11-2778 ###
### https://arxiv.org/pdf/cond-mat/0101373.pdf ###

### <span style='color:orange'>  Esta parte se basa en resultados obtenidos en PRB ###

-----------------------------------------------

#### NOTACIÓN PARA TODO EL CODIGO ####

Está calculado en forma matricial. En la tabla se resume cómo relacionar la notación de PRB
con el código. 

|   PRB  | Código  | ¿se integra? |  salida |
| :---:  |  :---:  |     :---:    |  :---:  |
| **u**  | (ux,uy) |       no     |(ux,uy) o (ths,phs)|
| **p1** | (rx,ry) |       sí     |   ----  |
| **p2** | (tx,ty) |       sí     |   ----  |
| **p0** | (qx,qy) |       no     |(qx, qy) o (thi,phi)|


(ux,uy): onda dispersada en cartesianas
    
(ths,phs): ángulos de la onda dispersada
    

(qx,qy): onda incidente en cartesianas
    
(thi,phi): ángulos de la onda incidente

In [1]:
import numpy as np
from numpy import cos, sin, pi, log10, exp, sqrt, prod

<h2> Defino kx, ky y kz(ep) </h2>

In [2]:
def kx(k,th,ph):
    return k*sin(th)*cos(ph)

def ky(k,th,ph):
    return k*sin(th)*sin(ph)

def alpha(k,x,y,ep):
    return sqrt(ep*k**2-x**2-y**2 +0j)


<h2> Defino producto entre matrices y matriz inversa para 2x2 </h2>

In [3]:
def matrix_prod(A,B):
    
    r00 = A[0,0]*B[0,0] + A[0,1]*B[1,0]
    r01 = A[0,0]*B[0,1] + A[0,1]*B[1,1]
    r10 = A[1,0]*B[0,0] + A[1,1]*B[1,0]
    r11 = A[1,0]*B[0,1] + A[1,1]*B[1,1]
    
    C = np.array([(r00,r01),(r10,r11)])
    
    return C


def matrix_inverse(A):
    det = A[0,0]*A[1,1]-A[0,1]*A[1,0]
    
    C = 1/det*np.array([(A[1,1],-A[0,1]),(-A[1,0],A[0,0])])
    
    return C


<h1> <span style='color:red'>  
     UNA SOLA INTERFAZ
    </span> </h1>

<h2> Defino matrices auxiliares D+-, Q+- y P </h2>

In [4]:
# -- Matrices D --

# D+

def Dmas(ux,uy,ki,ep0,ep1):
    
    d00 = ep1*alpha(ki,ux,uy,ep0) + ep0*alpha(ki,ux,uy,ep1)
    d01 = 0
    d10 = 0
    d11 = alpha(ki,ux,uy,ep0) + alpha(ki,ux,uy,ep1)
    
    dmas = np.array([(d00,d01),(d10,d11)])
    
    return dmas

# D-

def Dmenos(ux,uy,ki,ep0,ep1):
    
    d00 = ep1*alpha(ki,ux,uy,ep0) - ep0*alpha(ki,ux,uy,ep1)
    d01 = 0
    d10 = 0
    d11 = alpha(ki,ux,uy,ep0) - alpha(ki,ux,uy,ep1)
    
    dmenos = np.array([(d00,d01),(d10,d11)])
    
    return dmenos


# Inversas de D+-

def Inv_Dmas(ux,uy,ki,ep0,ep1):
    
    d00 = 1/(ep1*alpha(ki,ux,uy,ep0) + ep0*alpha(ki,ux,uy,ep1))
    d01 = 0
    d10 = 0
    d11 = 1/(alpha(ki,ux,uy,ep0) + alpha(ki,ux,uy,ep1))
    
    m = np.array([(d00,d01),(d10,d11)])
    
    return m



def Inv_Dmenos(ux,uy,ki,ep0,ep1):
    
    d00 = 1/(ep1*alpha(ki,ux,uy,ep0) - ep0*alpha(ki,ux,uy,ep1))
    d01 = 0
    d10 = 0
    d11 = 1/(alpha(ki,ux,uy,ep0) - alpha(ki,ux,uy,ep1))
    
    m = np.array([(d00,d01),(d10,d11)])
    
    return m



# -- Matrices Q --

# Q+(x,y)

def Qmas(x1,x2,y1,y2,ki,ep0,ep1):
    
    f = 0.5*(alpha(ki,x1,x2,ep1)-alpha(ki,x1,x2,ep0))/alpha(ki,y1,y2,ep0)
    
    X = np.sqrt(x1**2+x2**2)
    Y = np.sqrt(y1**2+y2**2)
    
    x_dot_y = (x1*y1+x2*y2)/(X*Y)
    x_X_y = (x1*y2-x2*y1)/(X*Y)
    
    D = X**2+alpha(ki,x1,x2,ep0)*alpha(ki,x1,x2,ep1)
    
    rh = (alpha(ki,y1,y2,ep0)-alpha(ki,y1,y2,ep1))/(alpha(ki,y1,y2,ep0)+alpha(ki,y1,y2,ep1))
    rv = (ep1*alpha(ki,y1,y2,ep0)-ep0*alpha(ki,y1,y2,ep1))/(ep1*alpha(ki,y1,y2,ep0)+ep0*alpha(ki,y1,y2,ep1))
    
    
    r00 = f*(X*Y*(1+rv)+(rv-1)*alpha(ki,y1,y2,ep0)*alpha(ki,x1,x2,ep1)*x_dot_y)/D
    r01 = -f*(1+rh)*ki*np.sqrt(ep0)*alpha(ki,x1,x2,ep1)*x_X_y/D
    r10 = f*(rv-1)*alpha(ki,y1,y2,ep0)*x_X_y/(ki*np.sqrt(ep0))
    r11 = f*(1+rh)*x_dot_y
    
    m = np.array([(r00,r01),(r10,r11)])
    
    return m


#Q-(x,y)

def Qmenos(x1,x2,y1,y2,ki,ep0,ep1):
    
    #try
    
    f = 0.5*(alpha(ki,x1,x2,ep1)-alpha(ki,x1,x2,ep0))/alpha(ki,y1,y2,ep0)
    
    X = np.sqrt(x1**2+x2**2)
    Y = np.sqrt(y1**2+y2**2)
    
    x_dot_y = (x1*y1+x2*y2)/(X*Y)
    x_X_y = (x1*y2-x2*y1)/(X*Y)
    
    D = X**2+alpha(ki,x1,x2,ep0)*alpha(ki,x1,x2,ep1)
    
    rh = (alpha(ki,y1,y2,ep0)-alpha(ki,y1,y2,ep1))/(alpha(ki,y1,y2,ep0)+alpha(ki,y1,y2,ep1))
    rv = (ep1*alpha(ki,y1,y2,ep0)-ep0*alpha(ki,y1,y2,ep1))/(ep1*alpha(ki,y1,y2,ep0)+ep0*alpha(ki,y1,y2,ep1))
    
    
    r00 = f*(X*Y*(1-rv)-(rv+1)*alpha(ki,y1,y2,ep0)*alpha(ki,x1,x2,ep1)*x_dot_y)/D
    r01 = -f*(1-rh)*ki*np.sqrt(ep0)*alpha(ki,x1,x2,ep1)*x_X_y/D
    r10 = -f*(rv+1)*alpha(ki,y1,y2,ep0)*x_X_y/(ki*np.sqrt(ep0))
    r11 = f*(1-rh)*x_dot_y
    
    m = np.array([(r00,r01),(r10,r11)])
    
    return m



# -- Matriz P --

def P(x1,x2,y1,y2,ki,ep0,ep1):
        
    f = alpha(ki,x1,x2,ep1)-alpha(ki,x1,x2,ep0)
    
    X = np.sqrt(x1**2+x2**2)
    Y = np.sqrt(y1**2+y2**2)
    
    x_dot_y = (x1*y1+x2*y2)/(X*Y)
    x_X_y = (x1*y2-x2*y1)/(X*Y)
    
    D = X**2+alpha(ki,x1,x2,ep0)*alpha(ki,x1,x2,ep1)
    
    r00 = f*(X*Y+alpha(ki,y1,y2,ep0)*alpha(ki,x1,x2,ep1)*x_dot_y)/D
    r01 = -f*ki*np.sqrt(ep0)*alpha(ki,x1,x2,ep1)*x_X_y/D
    r10 = f*alpha(ki,y1,y2,ep0)*x_X_y/(ki*np.sqrt(ep0))
    r11 = f*x_dot_y
    
    m = np.array([(r00,r01),(r10,r11)])
    
    return m


# Producto matricial P(x,r)·Q(r,y)

def P_Qmas(x1,x2,r1,r2,y1,y2,ki,ep0,ep1):
    
    m = matrix_prod(P(x1,x2,r1,r2,ki,ep0,ep1),Qmas(r1,r2,y1,y2,ki,ep0,ep1))
    
    return m

<h1> Amplitudes de Scattering </h1>

<h2> Orden CERO </h2>

In [5]:
def X0(qx,qy,ki,ep0,ep1):
    
    m00 = (ep1*alpha(ki,qx,qy,ep0) - ep0*alpha(ki,qx,qy,ep1))/( ep1*alpha(ki,qx,qy,ep0) + ep0*alpha(ki,qx,qy,ep1))
    m11 = (alpha(ki,qx,qy,ep0) - alpha(ki,qx,qy,ep1))/(alpha(ki,qx,qy,ep0) + alpha(ki,qx,qy,ep1))
  
    
    m = np.array([(m00,0),(0,m11)])
    
    return m

<h2> Orden UNO </h2>

In [6]:
def X1(ux,uy,qx,qy,ki,ep0,ep1):
    
              #Qmas(ux,uy,qx,qy,ki,ep0,ep1)
    vv = 2*1j*Qmas(ux,uy,qx,qy,ki,ep0,ep1)[0,0]
    vh = 2*1j*Qmas(ux,uy,qx,qy,ki,ep0,ep1)[0,1]
    hv = 2*1j*Qmas(ux,uy,qx,qy,ki,ep0,ep1)[1,0]
    hh = 2*1j*Qmas(ux,uy,qx,qy,ki,ep0,ep1)[1,1]
    
    aux = np.array([(vv,vh),(hv,hh)])
    
    return aux

<h2> Orden DOS </h2>

In [7]:
def X2(ux,uy,rx,ry,qx,qy,ki,ep0,ep1):
    
    vv = alpha(ki,ux,uy,ep1)*Qmas(ux,uy,qx,qy,ki,ep0,ep1)[0,0]+\
         alpha(ki,qx,qy,ep0)*Qmenos(ux,uy,qx,qy,ki,ep0,ep1)[0,0]-\
         2*P_Qmas(ux,uy,rx,ry,qx,qy,ki,ep0,ep1)[0,0]
    
    vh = alpha(ki,ux,uy,ep1)*Qmas(ux,uy,qx,qy,ki,ep0,ep1)[0,1]+\
         alpha(ki,qx,qy,ep0)*Qmenos(ux,uy,qx,qy,ki,ep0,ep1)[0,1]-\
         2*P_Qmas(ux,uy,rx,ry,qx,qy,ki,ep0,ep1)[0,1]
    
    hv = alpha(ki,ux,uy,ep1)*Qmas(ux,uy,qx,qy,ki,ep0,ep1)[1,0]+\
         alpha(ki,qx,qy,ep0)*Qmenos(ux,uy,qx,qy,ki,ep0,ep1)[1,0]-\
         2*P_Qmas(ux,uy,rx,ry,qx,qy,ki,ep0,ep1)[1,0]
    
    hh = alpha(ki,ux,uy,ep1)*Qmas(ux,uy,qx,qy,ki,ep0,ep1)[1,1]+\
         alpha(ki,qx,qy,ep0)*Qmenos(ux,uy,qx,qy,ki,ep0,ep1)[1,1]-\
         2*P_Qmas(ux,uy,rx,ry,qx,qy,ki,ep0,ep1)[1,1]
    
    
    aux = np.array([(vv,vh),(hv,hh)])
    
    return aux

<h2> Orden TRES </h2>

In [8]:
def X3(ux,uy,rx,ry,tx,ty,qx,qy,ki,ep0,ep1):
    
    t1 = -1j/3*((alpha(ki,ux,uy,ep1)**2 + alpha(ki,qx,qy,ep0)**2)*Qmas(ux,uy,qx,qy,ki,ep0,ep1)+\
                2*alpha(ki,ux,uy,ep1)*alpha(ki,qx,qy,ep0)*Qmenos(ux,uy,qx,qy,ki,ep0,ep1))
    t2 = 1j*matrix_prod(P(ux,uy,rx,ry,ki,ep0,ep1),X2(rx,ry,tx,ty,qx,qy,ki,ep0,ep1))
    t3 = 1j*(alpha(ki,ux,uy,ep1)-alpha(ki,tx,ty,ep0))*P_Qmas(ux,uy,tx,ty,qx,qy,ki,ep0,ep1)
    #matrix_prod(P(ux,uy,tx,ty,ki,ep0,ep1),Qmas(tx,ty,qx,qy,ki,ep0,ep1))
    
    vv = t1[0,0] + t2[0,0] + t3[0,0]
    vh = t1[0,1] + t2[0,1] + t3[0,1]
    hv = t1[1,0] + t2[1,0] + t3[1,0]
    hh = t1[1,1] + t2[1,1] + t3[1,1]
    
    t = np.array([(vv,vh),(hv,hh)])
    
    return t

<h1> Espectro de rugosidad </h1>

In [9]:
def W(kx,ky,s1,l1):
    
    # gaussiano
    return pi*s1**2*l1**2*np.exp(-0.25*(kx**2+ky**2)*l1**2)

<h1> Secciones eficaces </h1>

<h2> Orden UNO </h2>

$$ I^{(1-1)}(\mathbf{u,q}) = \frac{k_i^4\,\cos(\theta_s)^2\,\cos(\theta_i)}{(2\pi)^2}\,W(\mathbf{u-q})\,\,|X^{(1)}(\mathbf{u},\mathbf{q})|^2$$

In [10]:
def S1_QP(ki,ths,phs,thi,phi,ep0,ep1,s1,l1):
    
    #qp = X1(ki,ths,phs,thi,phi,ep0,ep1)
    
    ksx = kx(ki,ths,phs)
    ksy = ky(ki,ths,phs)
    
    kix = kx(ki,thi,phi)
    kiy = ky(ki,thi,phi)
    
    qp = X1(ksx,ksy,kix,kiy,ki,ep0,ep1)
    
    DKx = ksx-kix
    DKy = ksy-kiy
    
    return ki**4*cos(ths)**2*cos(thi)/(2*pi)**2*np.abs(qp)**2*W(DKx,DKy,s1,l1)

<h2> Orden DOS </h2>

Deben ser integradas sobre los argumentos $[rx,ry]$ en $(-\infty;\infty)$


La amplitud de scattering de segundo orden se escribe como:

$$ I^{(2-2)}(\mathbf{u},\mathbf{q}) = \frac{ki^4\,\cos(\theta_s)^2\,\cos(\theta_i)}{(2\pi)^2}\,\iint\frac{d^2r}{(2\pi)^2}W(\mathbf{u}-\mathbf{r})\,W(\mathbf{r}-\mathbf{q})\,X^{(2)}(\mathbf{u},\mathbf{r},\mathbf{q})\,
\left[X^{(2)*}(\mathbf{u},\mathbf{r},\mathbf{q}) + X^{(2)*}(\mathbf{u},\mathbf{u}+\mathbf{q}-\mathbf{r},\mathbf{q})\right]
$$

donde $\mathbf{u}$ es el modo de la onda dispersada y $\mathbf{q}$ es el modo de la onda incidente.


In [11]:
def I2_qp(rx,ry,ki,ths,phs,thi,phi,ep0,ep1,s1,l1):
    
    kix = kx(ki,thi,phi)
    kiy = ky(ki,thi,phi)
    
    ksx = kx(ki,ths,phs)
    ksy = ky(ki,ths,phs)
    
    DKx_ri = ry-kix
    DKy_ri = ry-kiy

    DKx_sr = ksx-rx
    DKy_sr = ksy-ry    
    
    DKX = ksx+kix-rx
    DKY = ksy+kiy-ry
    
         # X2(ux,uy,rx,ry,qx,qy,ki,ep0,ep1)
    qp_a = X2(ksx,ksy,rx,ry,kix,kiy,ki,ep0,ep1)
    qp_b = X2(ksx,ksy,DKX,DKY,kix,kiy,ki,ep0,ep1)
    
    t1 = np.abs(qp_a)**2
    t2 = qp_a*np.conj(qp_b)
    
    return ki**4*cos(ths)**2*cos(thi)/(2*pi)**2*W(DKx_sr,DKy_sr,s1,l1)*W(DKx_ri,DKy_ri,s1,l1)*(t1+t2)

<h2> Orden TRES </h2>


Deben ser integradas sobre los argumentos $[rx,ry]$ en $(-\infty;\infty)$


La amplitud de scattering de tercer orden se escribe como:

$$ I^{(3-1)}(\mathbf{u},\mathbf{q}) = \frac{ki^4\,\cos(\theta_s)^2\,\cos(\theta_i)}{(2\pi)^2}\,2\Re \left\lbrace W(\mathbf{u}-\mathbf{q})\,X^{(1)}(\mathbf{u},\mathbf{q})\\
\iint\frac{d^2r}{(2\pi)^2}W(\mathbf{u}-\mathbf{r})\,W(\mathbf{r}-\mathbf{q})\,X^{(3)}(\mathbf{u},\mathbf{q},\mathbf{r},\mathbf{q})\,
\left[X^{(3)*}(\mathbf{u},\mathbf{r},\mathbf{q-u+r},\mathbf{q}) + X^{(3)*}(\mathbf{u},\mathbf{r},\mathbf{r},\mathbf{q})\right] \right\rbrace
$$

donde $\mathbf{u}$ es el modo de la onda dispersada y $\mathbf{q}$ es el modo de la onda incidente.


In [12]:
def I3_qp(rx,ry,ki,ths,phs,thi,phi,ep0,ep1,s1,l1):
    
    kix = kx(ki,thi,phi)
    kiy = ky(ki,thi,phi)
    
    ksx = kx(ki,ths,phs)
    ksy = ky(ki,ths,phs)
    
    DKx_si = ksx-kix
    DKy_si = ksy-kiy
    
    DKx_ri = ry-kix
    DKy_ri = ry-kiy

    DKx_sr = ksx-rx
    DKy_sr = ksy-ry    
    
    DKX = kix-ksx+rx
    DKY = kiy-ksy+ry
    
    t1 = W(DKx_ri,DKy_ri,s1,l1)*X3(ksx,ksy,kix,kiy,rx,ry,kix,kiy,ki,ep0,ep1)
    t2 = W(DKx_sr,DKy_sr,s1,l1)*(X3(ksx,ksy,rx,ry,DKX,DKY,kix,kiy,ki,ep0,ep1)+\
                                X3(ksx,ksy,rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1))
    
    return ki**4*cos(ths)**2*cos(thi)/(2*pi)**2*(t1+t2)

<h1> <span style='color:red'>  
     MEDIOS ESTRATIFICADOS 
    </span> </h1>

# ***Rough surface on the bottom side***

### Defino matrices auxiliares T10, V10, VH21 y U0 ###

In [2]:
# -- Matriz T10 --

def T10(ux,uy,ki,ep0,ep1):
    
    r00 = 2*alpha(ki,ux,uy,ep1)*sqrt(ep0*ep1)/(ep1*alpha(ki,ux,uy,ep0)+ep0*alpha(ki,ux,uy,ep1))
    r10 = 0
    r01 = 0
    r11 = 2*alpha(ki,ux,uy,ep1)/(alpha(ki,ux,uy,ep0)+alpha(ki,ux,uy,ep1))
    
    m = np.array([(r00, r01),(r10,r11)])
    
    return m



# -- Matriz V10 --

def V10(ux,uy,ki,ep0,ep1):
    
    r00 = (ep1*alpha(ki,ux,uy,ep0)-ep0*alpha(ki,ux,uy,ep1))/(ep1*alpha(ki,ux,uy,ep0)+ep0*alpha(ki,ux,uy,ep1))
    r01 = 0
    r10 = 0
    r11 = (alpha(ki,ux,uy,ep0)-alpha(ki,ux,uy,ep1))/(alpha(ki,ux,uy,ep0)+alpha(ki,ux,uy,ep1))
    
    m = np.array([(r00,r01),(r10,r11)])
    
    return m


# -- Matriz VH21 --

def VH21(ux,uy,qx,qy,ki,d,ep1,ep2):
    
    f = np.exp(2*1j*alpha(ki,ux,uy,ep1)*d)
    r00 = f*(ep2*alpha(ki,qx,qy,ep1)-ep1*alpha(ki,qx,qy,ep2))/(ep2*alpha(ki,qx,qy,ep1)+ep1*alpha(ki,qx,qy,ep2))
    r01 = 0
    r10 = 0
    r11 = f*(alpha(ki,qx,qy,ep1)-alpha(ki,qx,qy,ep2))/(alpha(ki,qx,qy,ep1)+alpha(ki,qx,qy,ep2))
    
    m = np.array([(r00,r01),(r10,r11)])
    
    return m


def U0(ux,uy,qx,qy,ki,ep0,ep1,ep2,d):
    
    f = np.exp(2*1j*alpha(ki,ux,uy,ep1)*d)
    
    rv10 = (ep1*alpha(ki,qx,qy,ep0)-ep0*alpha(ki,qx,qy,ep1))/(ep1*alpha(ki,qx,qy,ep0)+ep0*alpha(ki,qx,qy,ep1))
    rh10 = (alpha(ki,qx,qy,ep0)-alpha(ki,qx,qy,ep1))/(alpha(ki,qx,qy,ep0)+alpha(ki,qx,qy,ep1))
    
    rv21 = (ep2*alpha(ki,qx,qy,ep1)-ep1*alpha(ki,qx,qy,ep2))/(ep2*alpha(ki,qx,qy,ep1)+ep1*alpha(ki,qx,qy,ep2))
    rh21 = (alpha(ki,qx,qy,ep1)-alpha(ki,qx,qy,ep2))/(alpha(ki,qx,qy,ep1)+alpha(ki,qx,qy,ep2))
    
    m00 = 1/(1+f*rv10*rv21)
    m01 = 0
    m10 = 0
    m11 = 1/(1+f*rh10*rh21)
    
    m = np.array([(m00,m01),(m10,m11)])
    
    return m

# def U0(ux,uy,qx,qy,ki,ep0,ep1,ep2,d):
    
#     f = np.exp(2*1j*alpha(ki,ux,uy,ep1)*d)
    
#     rv10 = (ep1*alpha(ki,ux,uy,ep0)-ep0*alpha(ki,ux,uy,ep1))/(ep1*alpha(ki,ux,uy,ep0)+ep0*alpha(ki,ux,uy,ep1))
#     rh10 = (alpha(ki,ux,uy,ep0)-alpha(ki,ux,uy,ep1))/(alpha(ki,ux,uy,ep0)+alpha(ki,ux,uy,ep1))
    
#     rv21 = (ep2*alpha(ki,qx,qy,ep1)-ep1*alpha(ki,qx,qy,ep2))/(ep2*alpha(ki,qx,qy,ep1)+ep1*alpha(ki,qx,qy,ep2))
#     rh21 = (alpha(ki,qx,qy,ep1)-alpha(ki,qx,qy,ep2))/(alpha(ki,qx,qy,ep1)+alpha(ki,qx,qy,ep2))
    
#     m00 = 1/(1+f*rv10*rv21)
#     m01 = 0
#     m10 = 0
#     m11 = 1/(1+f*rh10*rh21)
    
#     m = np.array([(m00,m01),(m10,m11)])
    
#     return m

In [2]:
# defino producto de matrices que necesito para calcular X2_b

# def T10_p_U0_p(ux,uy,ki,ep0,ep1,ep2,d):
    
#     m = matrix_prod(T10(ux,uy,ki,ep0,ep1),U0(ux,uy,ux,uy,ki,ep0,ep1,ep2,d))
    
#     return m

def T10p_U0p(ux,uy,qx,qy,ki,ep0,ep1,ep2,d):
    
    rv10 = (ep1*alpha(ki,qx,qy,ep0)-ep0*alpha(ki,qx,qy,ep1))/(ep1*alpha(ki,qx,qy,ep0)+ep0*alpha(ki,qx,qy,ep1))
    rh10 = (alpha(ki,qx,qy,ep0)-alpha(ki,qx,qy,ep1))/(alpha(ki,qx,qy,ep0)+alpha(ki,qx,qy,ep1))
    
    rv21 = (ep2*alpha(ki,qx,qy,ep1)-ep1*alpha(ki,qx,qy,ep2))/(ep2*alpha(ki,qx,qy,ep1)+ep1*alpha(ki,qx,qy,ep2))
    rh21 = (alpha(ki,qx,qy,ep1)-alpha(ki,qx,qy,ep2))/(alpha(ki,qx,qy,ep1)+alpha(ki,qx,qy,ep2))
    
    f = np.exp(1j*2*d*alpha(ki,ux,uy,ep1))
    
    z11 = 2*np.sqrt(ep0*ep1)*alpha(ki,ux,uy,ep1)/(ep1*alpha(ki,ux,uy,ep0)+ep0*alpha(ki,ux,uy,ep1))*\
        1/(rv10*rv21*f+1)
    
    z22 = 2*alpha(ki,ux,uy,ep1)/(alpha(ki,ux,uy,ep0)+alpha(ki,ux,uy,ep1))*1/(rh10*rh21*f+1)
    
    m = np.array([(z11,0),(0,z22)])
    
    return m

## NOTA: U0(u,q) y T10(u) son matrices diagonales ==> U0(u,q).T10(u) = T10(u).U0(u,q)


# def U0_p1_V10_p1(rx,ry,ki,ep0,ep1,ep2,d):
    
#     m = matrix_prod(U0(rx,ry,rx,ry,ki,ep0,ep1,ep2,d),V10(rx,ry,ki,ep0,ep1))
    
#     return m

def U0p_V10p(ux,uy,qx,qy,ki,ep0,ep1,ep2,d):
    
    rv_u = (ep1*alpha(ki,ux,uy,ep0)-ep0*alpha(ki,ux,uy,ep1))/(ep1*alpha(ki,ux,uy,ep0)+ep0*alpha(ki,ux,uy,ep1))
    rh_u = (alpha(ki,ux,uy,ep0)-alpha(ki,ux,uy,ep1))/(alpha(ki,ux,uy,ep0)+alpha(ki,ux,uy,ep1))
    
    rv10 = (ep1*alpha(ki,qx,qy,ep0)-ep0*alpha(ki,qx,qy,ep1))/(ep1*alpha(ki,qx,qy,ep0)+ep0*alpha(ki,qx,qy,ep1))
    rh10 = (alpha(ki,qx,qy,ep0)-alpha(ki,qx,qy,ep1))/(alpha(ki,qx,qy,ep0)+alpha(ki,qx,qy,ep1))
    
    rv21 = (ep2*alpha(ki,qx,qy,ep1)-ep1*alpha(ki,qx,qy,ep2))/(ep2*alpha(ki,qx,qy,ep1)+ep1*alpha(ki,qx,qy,ep2))
    rh21 = (alpha(ki,qx,qy,ep1)-alpha(ki,qx,qy,ep2))/(alpha(ki,qx,qy,ep1)+alpha(ki,qx,qy,ep2))
    
    f = np.exp(1j*2*d*alpha(ki,ux,uy,ep1))
    
    m11 = rv_u/(rv10*rv21*f+1)
    m22 = rh_u/(rh10*rh21*f+1)
    
    m = np.array([(m11,0),(0,m22)])
    
    return m

<h3> Coeficiente de Orden CERO </h3>

In [14]:
def X0_b(qx,qy,ki,ep0,ep1,ep2,d):
    
    # iterface ep0-ep1
    r00 = (ep1*alpha(ki,qx,qy,ep0)-ep0*alpha(ki,qx,qy,ep1))/(ep1*alpha(ki,qx,qy,ep0)+ep0*alpha(ki,qx,qy,ep1))
    r11 = (alpha(ki,qx,qy,ep0)-alpha(ki,qx,qy,ep1))/(alpha(ki,qx,qy,ep0)+alpha(ki,qx,qy,ep1))
    
    # interface ep1-ep2
    f = np.exp(2*1j*alpha(ki,qx,qy,ep1)*d)
    s00 = f*(ep2*alpha(ki,qx,qy,ep1)-ep1*alpha(ki,qx,qy,ep2))/(ep2*alpha(ki,qx,qy,ep1)+ep1*alpha(ki,qx,qy,ep2))
    s11 = f*(alpha(ki,qx,qy,ep1)-alpha(ki,qx,qy,ep2))/(alpha(ki,qx,qy,ep1)+alpha(ki,qx,qy,ep2))
    
    # producto
    a = (r00+f*s00)/(1+f*r00*s00)
    b = (r11+f*s11)/(1+f*r11*s11)
    
    m = np.array([(a,0),(0,b)])
    
    return m
    

<h3> Coeficiente de Orden UNO </h3>

In [15]:
# def X1_b(ux,uy,qx,qy,ki,ep0,ep1,ep2,d):
    
#     X1 = 2*1j*Qmas(ux,uy,qx,qy,ki,ep1,ep2)
    
    
#     m1 = matrix_prod(T10(ux,uy,ki,ep0,ep1),U0(ux,uy,qx,qy,ki,ep0,ep1,ep2,d))
#     m2 = matrix_prod(m1,X1)
#     m3 = matrix_prod(m2,U0(qx,qy,qx,qy,ki,ep0,ep1,ep2,d))
#     m4 = matrix_prod(m3,T10(qx,qy,ki,ep0,ep1)) 
    
#     m = np.exp(1j*d*(alpha(ki,ux,uy,ep1)+alpha(ki,qx,qy,ep1)))*m4
    
#     return m

def X1_b(ux,uy,qx,qy,ki,ep0,ep1,ep2,d):
    
    X1 = 2*1j*Qmas(ux,uy,qx,qy,ki,ep1,ep2)
    f_uq = np.exp(1j*d*(alpha(ki,ux,uy,ep1)+alpha(ki,qx,qy,ep1)))
    
    m1 = f_uq*matrix_prod(T10p_U0p(ux,uy,ux,uy,ki,ep0,ep1,ep2,d),X1)
    m = matrix_prod(m1,T10p_U0p(qx,qy,qx,qy,ki,ep0,ep1,ep2,d))
    
    return m

<h3> Coeficiente de Orden DOS </h3>

In [5]:
def X2_b(ux,uy,rx,ry,qx,qy,ki,ep0,ep1,ep2,d):
    
    
    f_uq = np.exp(1j*d*(alpha(ki,ux,uy,ep1)+alpha(ki,qx,qy,ep1))) #p--->p0
    f_ur = np.exp(1j*d*(alpha(ki,ux,uy,ep1)+alpha(ki,rx,ry,ep1))) #p--->p1
    f_rq = np.exp(1j*d*(alpha(ki,rx,ry,ep1)+alpha(ki,qx,qy,ep1))) #p1--->p0
    
    m1 = matrix_prod(T10p_U0p(ux,uy,qx,qy,ki,ep0,ep1,ep2,d),X2(ux,uy,rx,ry,qx,qy,ki,ep1,ep2))
    t1 = f_uq*matrix_prod(m1,T10p_U0p(qx,qy,qx,qy,ki,ep0,ep1,ep2,d))
    
    m2 = f_ur*matrix_prod(T10p_U0p(ux,uy,ux,uy,ki,ep0,ep1,ep2,d),X1(ux,uy,rx,ry,ki,ep1,ep2))
    m3 = f_rq*matrix_prod(U0p_V10p(rx,ry,rx,ry,ki,ep0,ep1,ep2,d),X1(rx,ry,qx,qy,ki,ep1,ep2))
    m4 = matrix_prod(m2,m3)
    t2 = matrix_prod(m4,T10p_U0p(qx,qy,qx,qy,ki,ep0,ep1,ep2,d))
    
    
    return t1 - alpha(ki,rx,ry,ep1)*t2


<h3> Coeficiente de Orden TRES </h3>

In [17]:
def s_1(ux,uy,rx,ry,tx,ty,qx,qy,ki,ep0,ep1,ep2,d):
    
    f_uq = np.exp(1j*d*(alpha(ki,ux,uy,ep1)+alpha(ki,qx,qy,ep0)))
    
    return f_uq*X3(ux,uy,rx,ry,tx,ty,qx,qy,ki,ep1,ep2)



def s_2(ux,uy,rx,ry,tx,ty,qx,qy,ki,ep0,ep1,ep2,d):
    
    f_ut = np.exp(1j*d*(alpha(ki,ux,uy,ep1)+alpha(ki,tx,ty,ep1))) # p--->p2
    f_tq = np.exp(1j*d*(alpha(ki,tx,ty,ep1)+alpha(ki,qx,qy,ep1))) # p2--->p0
    
    m1 = matrix_prod(X2(ux,uy,rx,ry,tx,ty,ki,ep1,ep2),U0p_V10p(tx,ty,qx,qy,ki,ep0,ep1,ep2,d))
    m = -alpha(ki,tx,ty,ep1)*f_ut*f_tq*matrix_prod(m1,X1(tx,ty,qx,qy,ki,ep1,ep2))
    
    return m



def s_3(ux,uy,rx,ry,tx,ty,qx,qy,ki,ep0,ep1,ep2,d):
    
    f_ur = np.exp(1j*d*(alpha(ki,ux,uy,ep1)+alpha(ki,rx,ry,ep1))) # p--->p1
    f_rq = np.exp(1j*d*(alpha(ki,rx,ry,ep1)+alpha(ki,qx,qy,ep1))) # p1--->p0
        
    m1 = matrix_prod(X1(ux,uy,rx,ry,ki,ep1,ep2),U0p_V10p(rx,ry,qx,qy,ki,ep0,ep1,ep2,d))
    m = -alpha(ki,rx,ry,ep1)*f_ur*f_rq*matrix_prod(m1,X2(rx,ry,tx,ty,qx,qy,ki,ep1,ep2))
    
    return m


  
def s_4(ux,uy,rx,ry,tx,ty,qx,qy,ki,ep0,ep1,ep2,d):
    
        
    f_ur = np.exp(1j*d*(alpha(ki,ux,uy,ep1)+alpha(ki,rx,ry,ep1))) # p--->p1
    f_rt = np.exp(1j*d*(alpha(ki,rx,ry,ep1)+alpha(ki,tx,ty,ep1))) # p1--->p2
    f_tq = np.exp(1j*d*(alpha(ki,tx,ty,ep1)+alpha(ki,qx,qy,ep1))) # p2--->p0
        
    m1 = matrix_prod(X1(ux,uy,rx,ry,ki,ep1,ep2),U0p_V10p(rx,ry,qx,qy,ki,ep0,ep1,ep2,d))
    m2 = matrix_prod(m1,X1(rx,ry,tx,ty,ki,ep1,ep2))
    m3 = matrix_prod(m2,U0p_V10p(tx,ty,qx,qy,ki,ep0,ep1,ep2,d))
    m = alpha(ki,rx,ry,ep1)*alpha(ki,tx,ty,ep1)*f_ur*f_rt*f_tq*matrix_prod(m3,X1(tx,ty,qx,qy,ki,ep1,ep2))
    
    return m
    


def X3_b(ux,uy,rx,ry,tx,ty,qx,qy,ki,ep0,ep1,ep2,d):
    
    S = s_1(ux,uy,rx,ry,tx,ty,qx,qy,ki,ep0,ep1,ep2,d) + s_2(ux,uy,rx,ry,tx,ty,qx,qy,ki,ep0,ep1,ep2,d)+\
        s_3(ux,uy,rx,ry,tx,ty,qx,qy,ki,ep0,ep1,ep2,d) + s_4(ux,uy,rx,ry,tx,ty,qx,qy,ki,ep0,ep1,ep2,d)
    
    m1 = matrix_prod(T10p_U0p(ux,uy,qx,qy,ki,ep0,ep1,ep2,d),S)
    m = matrix_prod(m1,T10p_U0p(qx,qy,qx,qy,ki,ep0,ep1,ep2,d))
    
    return m

# Secciones eficaces #

## Orden UNO ##

$$ I^{(1-1)}(\mathbf{u},\mathbf{q}) = \frac{k_i^4\,\cos(\theta_s)^2\,\cos(\theta_i)}{(2\pi)^2}\,W(\mathbf{u-q})\,\,|X^{(1)}(\mathbf{u},\mathbf{q})|^2$$

In [ ]:
def S1_QP_b(ki,ths,phs,thi,phi,e0,e1,e2,d,s,l):
    
    ksx = kx(ki,ths,phs)
    ksy = ky(ki,ths,phs)
    
    kix = kx(ki,thi,phi)
    kiy = ky(ki,thi,phi)
    
    DKx_si = ksx-kix
    DKy_si = ksy-kiy
    
    f = ki**4*cos(ths)**2*cos(thi)/(2*pi)**2*W(DKx_si,DKy_si,s,l)
    
    vv = f*np.abs(X1_b(ksx,ksy,kix,kiy,ki,e0,e1,e2,d)[0,0])**2
    hv = f*np.abs(X1_b(ksx,ksy,kix,kiy,ki,e0,e1,e2,d)[0,1])**2
    vh = f*np.abs(X1_b(ksx,ksy,kix,kiy,ki,e0,e1,e2,d)[1,0])**2
    hh = f*np.abs(X1_b(ksx,ksy,kix,kiy,ki,e0,e1,e2,d)[1,1])**2
    
    t = np.array([(vv,hv),(vh,hh)])
    
    return t

## Orden DOS ##

Deben ser integradas sobre los argumentos $[rx,ry]$ en $(-\infty;\infty)$


La amplitud de scattering de segundo orden se escribe como:

$$ I^{(2-2)}(\mathbf{u},\mathbf{q}) = \frac{ki^4\,\cos(\theta_s)^2\,\cos(\theta_i)}{(2\pi)^2}\,\iint\frac{d^2r}{(2\pi)^2}W(\mathbf{u}-\mathbf{r})\,W(\mathbf{r}-\mathbf{q})\,X^{(2)}(\mathbf{u},\mathbf{r},\mathbf{q})\,
\left[X^{(2)*}(\mathbf{u},\mathbf{r},\mathbf{q}) + X^{(2)*}(\mathbf{u},\mathbf{u}+\mathbf{q}-\mathbf{r},\mathbf{q})\right]
$$

donde $\mathbf{u}$ es el modo de la onda dispersada y $\mathbf{q}$ es el modo de la onda incidente.


In [ ]:
def I2_QP_b(rx,ry,ki,ths,phs,thi,phi,ep0,ep1,ep2,d,s,l):
    
    ksx = kx(ki,ths,phs)
    ksy = ky(ki,ths,phs)
    
    kix = kx(ki,thi,phi)
    kiy = ky(ki,thi,phi)
    
    DKx_sr = ksx-rx
    DKy_sr = ksy-ry
    
    DKx_ri = rx-kix
    DKy_ri = ry-kiy
    
    DKX = ksx+kix-rx
    DKY = ksy+kiy-ry
    
    WW = W(DKx_sr,DKy_sr,s,l)*W(DKx_ri,DKy_ri,s,l)
    
    t1 = X2_b(ksx,ksy,rx,ry,kix,kiy,ki,ep0,ep1,ep2,d)
    t2 = X2_b(ksx,ksy,DKX,DKY,kix,kiy,ki,ep0,ep1,ep2,d)
    
    out = WW*t1*np.conj(t1+t2)
    
    return out

## Orden TRES ##

Deben ser integradas sobre los argumentos $[rx,ry]$ en $(-\infty;\infty)$


La amplitud de scattering de tercer orden se escribe como:

$$ I^{(3-1)}(\mathbf{u},\mathbf{q}) = \frac{ki^4\,\cos(\theta_s)^2\,\cos(\theta_i)}{(2\pi)^2}\,2\,\Re\left\lbrace W(\mathbf{u}-\mathbf{q})\,X^{(1)}(\mathbf{u},\mathbf{q}) \\
 \iint\frac{d^2r}{(2\pi)^2}W(\mathbf{u}-\mathbf{r})\,W(\mathbf{r}-\mathbf{q})\,X^{(3)*}(\mathbf{u},\mathbf{q},\mathbf{r},\mathbf{q})\,
\left[X^{(3)*}(\mathbf{u},\mathbf{r},\mathbf{q-u+r},\mathbf{q}) + X^{(3)*}(\mathbf{u},\mathbf{r},\mathbf{r},\mathbf{q})\right]\right\rbrace
$$

donde $\mathbf{u}$ es el modo de la onda dispersada y $\mathbf{q}$ es el modo de la onda incidente.

In [ ]:
def I3_QP_b(rx,ry,ki,ths,phs,thi,phi,ep0,ep1,ep2,d,s,l):
    
    ksx = kx(ki,ths,phs)
    ksy = ky(ki,ths,phs)
    
    kix = kx(ki,thi,phi)
    kiy = ky(ki,thi,phi)
    
    DKx_sr = ksx-rx
    DKy_sr = ksy-ry
    
    DKx_ri = rx-kix
    DKy_ri = ry-kiy
    
    DKX = -ksx+kix+rx
    DKY = -ksy+kiy+ry
    
    WW = W(DKx_sr,DKy_sr,s,l)*W(DKx_ri,DKy_ri,s,l)
    
    t1 = X3_b(ksx,ksy,kix,kiy,rx,ry,kix,kiy,ki,ep0,ep1,ep2,d)
    t2 = X3_b(ksx,ksy,rx,ry,DKX,DKY,kix,kiy,ki,ep0,ep1,ep2,d)+X3_b(ksx,ksy,rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)
    
    out = WW*t1*np.conj(t2)
    
    return out

# ***Rough surface on the upper side***

### Defino matrices Fh y Fv ###

In [ ]:
def Fmas_v(ux,uy,qx,qy,ki,ep0,ep1,ep2,d):
    
    f = np.exp(2*1j*alpha(ki,ux,uy,ep1)*d)
    a21 = (ep2*alpha(ki,qx,qy,ep1)-ep1*alpha(ki,qx,qy,ep2))/(ep2*alpha(ki,qx,qy,ep1)+ep1*alpha(ki,qx,qy,ep2))
    a10 = (ep1*alpha(ki,qx,qy,ep0)-ep0*alpha(ki,qx,qy,ep1))/(ep1*alpha(ki,qx,qy,ep0)+ep0*alpha(ki,qx,qy,ep1))
    
    return (1+f*a21)/(1+f*a21*a10)


def Fmenos_v(ux,uy,qx,qy,ki,ep0,ep1,ep2,d):
    
    f = np.exp(2*1j*alpha(ki,ux,uy,ep1)*d)
    a21 = (ep2*alpha(ki,qx,qy,ep1)-ep1*alpha(ki,qx,qy,ep2))/(ep2*alpha(ki,qx,qy,ep1)+ep1*alpha(ki,qx,qy,ep2))
    a10 = (ep1*alpha(ki,qx,qy,ep0)-ep0*alpha(ki,qx,qy,ep1))/(ep1*alpha(ki,qx,qy,ep0)+ep0*alpha(ki,qx,qy,ep1))
    
    return (1-f*a21)/(1+f*a21*a10)


def Fmas_h(ux,uy,qx,qy,ki,ep0,ep1,ep2,d):
    
    f = np.exp(2*1j*alpha(ki,ux,uy,ep1)*d)
    b21 = (alpha(ki,qx,qy,ep1)-alpha(ki,qx,qy,ep2))/(alpha(ki,qx,qy,ep1)+alpha(ki,qx,qy,ep2))
    b10 = (alpha(ki,qx,qy,ep0)-alpha(ki,qx,qy,ep1))/(alpha(ki,qx,qy,ep0)+alpha(ki,qx,qy,ep1))
    
    return (1+f*b21)/(1+f*b21*b10)


def Fmenos_h(ux,uy,qx,qy,ki,ep0,ep1,ep2,d):
    
    f = np.exp(2*1j*alpha(ki,ux,uy,ep1)*d)
    b21 = (alpha(ki,qx,qy,ep1)-alpha(ki,qx,qy,ep2))/(alpha(ki,qx,qy,ep1)+alpha(ki,qx,qy,ep2))
    b10 = (alpha(ki,qx,qy,ep0)-alpha(ki,qx,qy,ep1))/(alpha(ki,qx,qy,ep0)+alpha(ki,qx,qy,ep1))
    
    return (1-f*b21)/(1+f*b21*b10)


### Defino  matrices Qab ###

In [ ]:
# # -- Matriz Q++ --

def Qmas_mas(ux,uy,qx,qy,ki,ep0,ep1,ep2,d):
    
    U = np.sqrt(ux**2+uy**2)
    Q = np.sqrt(qx**2+qy**2)
    
    a_u = ep0*alpha(ki,ux,uy,ep1)+ep1*alpha(ki,ux,uy,ep0)
    a_q = ep0*alpha(ki,qx,qy,ep1)+ep1*alpha(ki,qx,qy,ep0)
    
    b_u = alpha(ki,ux,uy,ep1)+alpha(ki,ux,uy,ep0)
    b_q = alpha(ki,qx,qy,ep1)+alpha(ki,qx,qy,ep0)
    
    u_dot_q = (ux*qx+uy*qy)/(U*Q)
    u_X_q = (ux*qy-uy*qx)/(U*Q)
    
    
    m00 = (ep1-ep0)/(a_q*a_u)*(ep1*U*Q*Fmas_v(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)*Fmas_v(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)-\
                              ep0*alpha(ki,ux,uy,ep1)*alpha(ki,qx,qy,ep1)*Fmenos_v(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)*Fmenos_v(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)*u_dot_q)
    
    m01 = (ep1-ep0)/(a_u*b_q)*(-np.sqrt(ep0)*ki*alpha(ki,ux,uy,ep1)*Fmenos_v(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)*Fmas_h(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)*u_X_q)
    
    m10 = (ep1-ep0)/(a_q*b_u)*(-np.sqrt(ep0)*ki*alpha(ki,qx,qy,ep1)*Fmas_h(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)*Fmenos_v(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)*u_X_q)
    
    m11 = (ep1-ep0)/(b_u*b_q)*ki**2*Fmas_h(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)*Fmas_h(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)*u_dot_q
    
    m = np.array([(m00,m01),(m10,m11)])
    
    return m


# -- Matriz Q-+ --

def Qmenos_mas(ux,uy,qx,qy,ki,ep0,ep1,ep2,d):
    

    U = np.sqrt(ux**2+uy**2)
    Q = np.sqrt(qx**2+qy**2)
    
    a_u = ep0*alpha(ki,ux,uy,ep1)+ep1*alpha(ki,ux,uy,ep0)
    a_q = ep0*alpha(ki,qx,qy,ep1)+ep1*alpha(ki,qx,qy,ep0)
    
    b_u = alpha(ki,ux,uy,ep1)+alpha(ki,ux,uy,ep0)
    b_q = alpha(ki,qx,qy,ep1)+alpha(ki,qx,qy,ep0)
    
    u_dot_q = (ux*qx+uy*qy)/(U*Q)
    u_X_q = (ux*qy-uy*qx)/(U*Q)
    
    
    m00 = (ep1-ep0)/(a_q*a_u)*(ep1*U*Q*Fmenos_v(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)*Fmas_v(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)-\
                              ep0*alpha(ki,ux,uy,ep1)*alpha(ki,qx,qy,ep1)*Fmas_v(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)*Fmenos_v(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)*u_dot_q)
    
    m01 = (ep1-ep0)/(a_u*b_q)*(-np.sqrt(ep0)*ki*alpha(ki,ux,uy,ep1)*Fmas_v(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)*Fmas_h(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)*u_X_q)
    
    m10 = (ep1-ep0)/(a_q*b_u)*(-np.sqrt(ep0)*ki*alpha(ki,qx,qy,ep1)*Fmenos_h(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)*Fmenos_v(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)*u_X_q)
    
    m11 = (ep1-ep0)/(b_u*b_q)*ki**2*Fmenos_h(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)*Fmas_h(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)*u_dot_q
    
    m = np.array([(m00,m01),(m10,m11)])
    
    return m



# --Matriz Q+- --

def Qmas_menos(ux,uy,qx,qy,ki,ep0,ep1,ep2,d):
    
    U = np.sqrt(ux**2+uy**2)
    Q = np.sqrt(qx**2+qy**2)
    
    a_u = ep0*alpha(ki,ux,uy,ep1)+ep1*alpha(ki,ux,uy,ep0)
    a_q = ep0*alpha(ki,qx,qy,ep1)+ep1*alpha(ki,qx,qy,ep0)
    
    b_u = alpha(ki,ux,uy,ep1)+alpha(ki,ux,uy,ep0)
    b_q = alpha(ki,qx,qy,ep1)+alpha(ki,qx,qy,ep0)
    
    u_dot_q = (ux*qx+uy*qy)/(U*Q)
    u_X_q = (ux*qy-uy*qx)/(U*Q)
    
    
    m00 = (ep1-ep0)/(alpha(ki,qx,qy,ep0)*a_q*a_u)*(ep0*alpha(ki,qx,qy,ep1)*U*Q*Fmas_v(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)*Fmenos_v(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)-\
                              ep1*alpha(ki,ux,uy,ep1)*alpha(ki,qx,qy,ep0)**2*Fmenos_v(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)*Fmas_v(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)*u_dot_q)
    
    m01 = (ep1-ep0)/(alpha(ki,qx,qy,ep0)*a_u*b_q)*(-np.sqrt(ep0)*ki*alpha(ki,ux,uy,ep1)*alpha(ki,qx,qy,ep1)*Fmenos_v(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)*Fmenos_h(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)*u_X_q)
    
    m10 = (ep1-ep0)/(alpha(ki,qx,qy,ep0)*a_q*b_u)*(-np.sqrt(ep0)*ep1*ki*alpha(ki,qx,qy,ep0)**2*Fmas_h(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)*Fmas_v(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)*u_X_q)
    
    m11 = (ep1-ep0)/(alpha(ki,qx,qy,ep0)*b_u*b_q)*ki**2*alpha(ki,qx,qy,ep1)*Fmas_h(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)*Fmenos_h(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)*u_dot_q
    
    m = np.array([(m00,m01),(m10,m11)])
    
    return m


# --Matriz Q-- --

def Qmenos_menos(ux,uy,qx,qy,ki,ep0,ep1,ep2,d):
    
        
    U = np.sqrt(ux**2+uy**2)
    Q = np.sqrt(qx**2+qy**2)
    
    a_u = ep0*alpha(ki,ux,uy,ep1)+ep1*alpha(ki,ux,uy,ep0)
    a_q = ep0*alpha(ki,qx,qy,ep1)+ep1*alpha(ki,qx,qy,ep0)
    
    b_u = alpha(ki,ux,uy,ep1)+alpha(ki,ux,uy,ep0)
    b_q = alpha(ki,qx,qy,ep1)+alpha(ki,qx,qy,ep0)
    
    u_dot_q = (ux*qx+uy*qy)/(U*Q)
    u_X_q = (ux*qy-uy*qx)/(U*Q)
    
    
    m00 = (ep1-ep0)/(alpha(ki,qx,qy,ep0)*a_q*a_u)*(ep0*alpha(ki,qx,qy,ep1)*U*Q*Fmenos_v(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)*Fmenos_v(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)-\
                              ep1*alpha(ki,ux,uy,ep1)*alpha(ki,qx,qy,ep0)**2*Fmas_v(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)*Fmas_v(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)*u_dot_q)
    
    m01 = (ep1-ep0)/(alpha(ki,qx,qy,ep0)*a_u*b_q)*(-np.sqrt(ep0)*ki*alpha(ki,ux,uy,ep1)*alpha(ki,qx,qy,ep1)*Fmas_v(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)*Fmenos_h(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)*u_X_q)
    
    m10 = (ep1-ep0)/(alpha(ki,qx,qy,ep0)*a_q*b_u)*(-np.sqrt(ep0)*ep1*ki*alpha(ki,qx,qy,ep0)**2*Fmenos_h(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)*Fmas_v(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)*u_X_q)
    
    m11 = (ep1-ep0)/(alpha(ki,qx,qy,ep0)*b_u*b_q)*ki**2*alpha(ki,qx,qy,ep1)*Fmenos_h(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)*Fmenos_h(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)*u_dot_q
    
    m = np.array([(m00,m01),(m10,m11)])
        
    return m

### Defino matrices P+ y P- ###

In [ ]:
# -- Matriz P+ --

def Pmas(ux,uy,qx,qy,ki,ep0,ep1,ep2,d):
    
    U = np.sqrt(ux**2+uy**2)
    Q = np.sqrt(qx**2+qy**2)
    
    u_dot_q = (ux*qx+uy*qy)/(U*Q)
    u_X_q = (ux*qy-uy*qx)/(U*Q)
    
    f00 = (ep1-ep0)/(ep1*alpha(ki,ux,uy,ep0) + ep0*alpha(ki,ux,uy,ep1))
   
    f11 = (ep1-ep0)/(alpha(ki,ux,uy,ep0) + alpha(ki,ux,uy,ep1))
    

    p00 = f00*(U*Q*Fmas_v(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)+alpha(ki,ux,uy,ep1)*alpha(ki,qx,qy,ep0)*Fmenos_v(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)*u_dot_q)
    
    p01 = -f00*np.sqrt(ep0)*ki*alpha(ki,ux,uy,ep1)*Fmenos_v(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)*u_X_q
    
    p10 = f11*np.sqrt(ep0)*ki*alpha(ki,qx,qy,ep0)*Fmas_h(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)*u_X_q
    
    p11 = f11*ki**2*Fmas_h(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)*u_dot_q
    
    p = np.array([(p00,p01),(p10,p11)])
    
    return p


# --Matrix P- --

def Pmenos(ux,uy,qx,qy,ki,ep0,ep1,ep2,d):
    
    U = np.sqrt(ux**2+uy**2)
    Q = np.sqrt(qx**2+qy**2)
    
    u_dot_q = (ux*qx+uy*qy)/(U*Q)
    u_X_q = (ux*qy-uy*qx)/(U*Q)
    
    f00 = (ep1-ep0)/(ep1*alpha(ki,ux,uy,ep0) + ep0*alpha(ki,ux,uy,ep1))
   
    f11 = (ep1-ep0)/(alpha(ki,ux,uy,ep0) + alpha(ki,ux,uy,ep1))
    

    p00 = f00*(U*Q*Fmenos_v(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)+alpha(ki,ux,uy,ep1)*alpha(ki,qx,qy,ep0)*Fmas_v(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)*u_dot_q)
    
    p01 = -f00*np.sqrt(ep0)*ki*alpha(ki,ux,uy,ep1)*Fmas_v(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)*u_X_q
    
    p10 = f11*np.sqrt(ep0)*ki*alpha(ki,qx,qy,ep0)*Fmenos_h(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)*u_X_q
    
    p11 = f11*ki**2*Fmenos_h(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)*u_dot_q
    
    p = np.array([(p00,p01),(p10,p11)])
    
    return p

In [ ]:
# Producto P(u,r).Q(r,q)

def Pmas_Qmasmas(rx,ry,ux,uy,qx,qy,ki,ep0,ep1,ep2,d):
    
    m = matrix_prod(Pmas(ux,uy,rx,ry,ki,ep0,ep1,ep2,d),Qmas_mas(rx,ry,qx,qy,ki,ep0,ep1,ep2,d))
    
    return m

def Pmenos_Qmasmas(rx,ry,ux,uy,qx,qy,ki,ep0,ep1,ep2,d):
    
    m = matrix_prod(Pmenos(ux,uy,rx,ry,ki,ep0,ep1,ep2,d),Qmas_mas(rx,ry,qx,qy,ki,ep0,ep1,ep2,d))
    
    return m

<h3> Coeficiente de Orden CERO </h3>

In [20]:
def X0_u(qx,qy,ki,ep0,ep1,ep2,d):
    
    # iterface ep0-ep1
    r00 = (ep1*alpha(ki,qx,qy,ep0)-ep0*alpha(ki,qx,qy,ep1))/(ep1*alpha(ki,qx,qy,ep0)+ep0*alpha(ki,qx,qy,ep1))
    r11 = (alpha(ki,qx,qy,ep0)-alpha(ki,qx,qy,ep1))/(alpha(ki,qx,qy,ep0)+alpha(ki,qx,qy,ep1))
    
    # interface ep1-ep2
    f = np.exp(2*1j*alpha(ki,qx,qy,ep1)*d)
    s00 = f*(ep2*alpha(ki,qx,qy,ep1)-ep1*alpha(ki,qx,qy,ep2))/(ep2*alpha(ki,qx,qy,ep1)+ep1*alpha(ki,qx,qy,ep2))
    s11 = f*(alpha(ki,qx,qy,ep1)-alpha(ki,qx,qy,ep2))/(alpha(ki,qx,qy,ep1)+alpha(ki,qx,qy,ep2))
    
    # producto
    a = (r00+f*s00)/(1+f*r00*s00)
    b = (r11+f*s11)/(1+f*r11*s11)
    
    m = np.array([(a,0),(0,b)])
    
    return m
    

<h3> Coeficiente de Orden UNO </h3>

In [21]:
# amplitud de scattring

def X1_u(ux,uy,qx,qy,ki,ep0,ep1,ep2,d):
    
                #Qmas_mas(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)
    return 2*1j*Qmas_mas(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)

<h3> Coeficiente de Orden DOS </h3>

In [22]:
def X2_uA(ux,uy,qx,qy,ki,ep0,ep1,ep2,d):
    
    t1 = alpha(ki,ux,uy,ep1)*Qmenos_mas(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)
    t2 = alpha(ki,qx,qy,ep0)*Qmas_menos(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)
    
    t = t1 + t2
    
    return t


def X2_uB(ux,uy,rx,ry,qx,qy,ki,ep0,ep1,ep2,d):
    
    t3 = -2*Pmas_Qmasmas(rx,ry,ux,uy,qx,qy,ki,ep0,ep1,ep2,d)
    
    return t3

<h3> Coeficiente de Orden TRES </h3>

In [1]:
def X3_uA(ux,uy,qx,qy,ki,ep0,ep1,ep2,d):
    
    t1 = (alpha(ki,ux,uy,ep1)**2 + alpha(ki,qx,qy,ep0)**2)*Qmas_mas(ux,uy,qx,qy,ki,ep0,ep1,ep2,d) + \
         2*alpha(ki,ux,uy,ep1)*alpha(ki,qx,qy,ep0)*Qmenos_menos(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)
    
    return -(1j/3)*t1



def X3_uB(ux,uy,tx,ty,qx,qy,ki,ep0,ep1,ep2,d):

    t1 = 1j*alpha(ki,ux,uy,ep1)*Pmenos_Qmasmas(tx,ty,ux,uy,qx,qy,ki,ep0,ep1,ep2,d)
    t2 = 1j*alpha(ki,tx,ty,ep0)*Pmas_Qmasmas(tx,ty,ux,uy,qx,qy,ki,ep0,ep1,ep2,d)
    
    return t1-t2



def X3_uC(ux,uy,rx,ry,tx,ty,qx,qy,ki,ep0,ep1,ep2,d):
    

    t2 = matrix_prod(Pmas(ux,uy,rx,ry,ki,ep0,ep1,ep2,d),X2(rx,ry,tx,ty,qx,qy,ki,ep0,ep1))

    
    return  1j*t2

# Secciones eficaces #

## Orden UNO ##

$$ I^{(1-1)}(\mathbf{u},\mathbf{q}) = \frac{k_i^4\,\cos(\theta_s)^2\,\cos(\theta_i)}{(2\pi)^2}\,W(\mathbf{u-q})\,\,|X^{(1)}(\mathbf{u},\mathbf{q})|^2$$

In [ ]:
def S1_QP_u(ki,ths,phs,thi,phi,ep0,ep1,ep2,d,l1,s1):
    
    ksx = kx(ki,ths,phs)
    ksy = ky(ki,ths,phs)
    
    kix = kx(ki,thi,phi)
    kiy = ky(ki,thi,phi)
    
    DKx_si = ksx-kix
    DKy_si = ksy-kiy
    
    f = ki**4*cos(ths)**2*cos(thi)/(2*pi)**2*W(DKx_si,DKy_si,s1,l1)
    qp = X1_u(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)
    
    return f*np.abs(qp)**2

## Orden DOS ##

Deben ser integradas sobre los argumentos $[rx,ry]$ en $(-\infty;\infty)$


La amplitud de scattering de segundo orden se escribe como:

$$ I^{(2-2)}(\mathbf{u},\mathbf{q}) = \frac{ki^4\,\cos(\theta_s)^2\,\cos(\theta_i)}{(2\pi)^2}\,\iint\frac{d^2r}{(2\pi)^2}W(\mathbf{u}-\mathbf{r})\,W(\mathbf{r}-\mathbf{q})\,X^{(2)}(\mathbf{u},\mathbf{r},\mathbf{q})\,
\left[X^{(2)*}(\mathbf{u},\mathbf{r},\mathbf{q}) + X^{(2)*}(\mathbf{u},\mathbf{u}+\mathbf{q}-\mathbf{r},\mathbf{q})\right]
$$

donde $\mathbf{u}$ es el modo de la onda dispersada y $\mathbf{q}$ es el modo de la onda incidente.

Pero la amplitud de scattering en este caso se escribe como la suma de dos términos:

$$X^{(2)}(\mathbf{u},\mathbf{r},\mathbf{q}) = X^{(2)}_a(\mathbf{u},\mathbf{r}) + X^{(2)}_b(\mathbf{u},\mathbf{r}, \mathbf{q}) $$

por lo que hay que expandir el integrando para que pueda hacerse de manera numérica la integral. 

Entonces, la amplitud de scattering nos queda:

$$ I^{(2-2)}(\mathbf{u},\mathbf{q}) = \frac{ki^4\,\cos(\theta_s)^2\,\cos(\theta_i)}{(2\pi)^2}\,\iint\frac{d^2r}{(2\pi)^2}W(\mathbf{u}-\mathbf{r})\,W(\mathbf{r}-\mathbf{q})\,
\left[2|X^{(2)}_a(\mathbf{u},\mathbf{q})|^2 + |X^{(2)}_b(\mathbf{u},\mathbf{r},\mathbf{q})|^2\\
+ 2\Re\left\lbrace X^{(2)}_a(\mathbf{u},\mathbf{q})\,\,X^{(2)*}_b(\mathbf{u},\mathbf{r},\mathbf{q})\right\rbrace +
X^{(2)}_a(\mathbf{u},\mathbf{q})\,\,X^{(2)*}_b(\mathbf{u},\mathbf{u+q-r},\mathbf{q})\\
+X^{(2)*}_a(\mathbf{u},\mathbf{q})\,\,X^{(2)}_b(\mathbf{u},\mathbf{r},\mathbf{q}) + 
X^{(2)}_b(\mathbf{u},\mathbf{r},\mathbf{q})\,\,X^{(2)*}_b(\mathbf{u},\mathbf{u+q-r},\mathbf{q})\right]
$$

In [ ]:
def I2_QP_u(rx,ry,ki,ths,phs,thi,phi,ep0,ep1,ep2,d,s1,l1):
    
    ksx = kx(ki,ths,phs)
    ksy = ky(ki,ths,phs)
    
    kix = kx(ki,thi,phi)
    kiy = ky(ki,thi,phi)
    
    DKx_sr = ksx-rx
    DKy_sr = ksy-ry
    
    DKx_ri = rx-kix
    DKy_ri = ry-kiy
    
    DKX = ksx+kix-rx
    DKY = ksy+kiy-ry
    
    f = ki**4*cos(ths)**2*cos(thi)/(2*pi)**2
    WW = W(DKx_sr,DKy_sr,s1,l1)*W(DKx_ri,DKy_ri,s1,l1)
    
    ## VV

    t1_vv = 2*np.abs(X2_uA(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,0])**2
    t2_vv = np.abs(X2_uB(rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,0])**2
    t3_vv = 2*np.real(X2_uA(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,0]*np.conj(X2_uB(rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,0]))
    t4_vv = X2_uA(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,0]*np.conj(X2_uB(DKX,DKY,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,0])
    t5_vv = np.conj(X2_uA(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,0])*X2_uB(rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,0]
    t6_vv = X2_uB(rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,0]*np.conj(X2_uB(DKX,DKY,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,0])
        
    vv = (t1_vv+t2_vv+t3_vv+t4_vv+t5_vv)*WW

    
    ## HV

    t1_hv = 2*np.abs(X2_uA(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,1])**2
    t2_hv = np.abs(X2_uB(rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,1])**2
    t3_hv = 2*np.real(X2_uA(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,1]*np.conj(X2_uB(rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,1]))
    t4_hv = X2_uA(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,1]*np.conj(X2_uB(DKX,DKY,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,1])
    t5_hv = np.conj(X2_uA(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,1])*X2_uB(rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,1]
    t6_hv = X2_uB(rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,1]*np.conj(X2_uB(DKX,DKY,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,1])
        
    hv = (t1_hv+t2_hv+t3_hv+t4_hv+t5_hv)*WW
    
    ## VH

    t1_vh = 2*np.abs(X2_uA(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,0])**2
    t2_vh = np.abs(X2_uB(rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,0])**2
    t3_vh = 2*np.real(X2_uA(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,0]*np.conj(X2_uB(rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,0]))
    t4_vh = X2_uA(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,0]*np.conj(X2_uB(DKX,DKY,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,0])
    t5_vh = np.conj(X2_uA(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,0])*X2_uB(rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,0]
    t6_vh = X2_uB(rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,0]*np.conj(X2_uB(DKX,DKY,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,0])
        
    vh = (t1_vh+t2_vh+t3_vh+t4_vh+t5_vh)*WW
    
    ## HH

    t1_hh = 2*np.abs(X2_uA(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,1])**2
    t2_hh = np.abs(X2_uB(rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,1])**2
    t3_hh = 2*np.real(X2_uA(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,1]*np.conj(X2_uB(rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,1]))
    t4_hh = X2_uA(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,1]*np.conj(X2_uB(DKX,DKY,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,1])
    t5_hh = np.conj(X2_uA(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,1])*X2_uB(rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,1]
    t6_hh = X2_uB(rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,1]*np.conj(X2_uB(DKX,DKY,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,1])
        
    hh = (t1_hh+t2_hh+t3_hh+t4_hh+t5_hh)*WW
    
    out = np.array([(vv,hv),(vh,hh)])
    
    return out

## Orden TRES ##

Deben ser integradas sobre los argumentos $[rx,ry]$ en $(-\infty;\infty)$


La amplitud de scattering de tercer orden se escribe como:

$$ I^{(3-1)}(\mathbf{u},\mathbf{q}) = \frac{ki^4\,\cos(\theta_s)^2\,\cos(\theta_i)}{(2\pi)^2}\,2\,\Re\left\lbrace W(\mathbf{u}-\mathbf{q})\,X^{(1)}(\mathbf{u},\mathbf{q}) \\
 \iint\frac{d^2r}{(2\pi)^2}W(\mathbf{u}-\mathbf{r})\,W(\mathbf{r}-\mathbf{q})\,X^{(3)*}(\mathbf{u},\mathbf{q},\mathbf{r},\mathbf{q})\,
\left[X^{(3)*}(\mathbf{u},\mathbf{r},\mathbf{q-u+r},\mathbf{q}) + X^{(3)*}(\mathbf{u},\mathbf{r},\mathbf{r},\mathbf{q})\right]\right\rbrace
$$

donde $\mathbf{u}$ es el modo de la onda dispersada y $\mathbf{q}$ es el modo de la onda incidente.

La amplitud de scattering en este caso se escribe como la suma de tres términos:

$$X^{(3)}(\mathbf{u},\mathbf{r},\mathbf{t},\mathbf{q}) = X^{(3)}_a(\mathbf{u},\mathbf{q})+
X^{(3)}_b(\mathbf{u},\mathbf{r},\mathbf{q}) + X^{(3)}_c(\mathbf{u},\mathbf{r},\mathbf{t},\mathbf{q})$$



Entonces, la amplitud de scattering nos queda:

$$ I^{(3-1)}(\mathbf{u},\mathbf{q}) = \frac{ki^4\,\cos(\theta_s)^2\,\cos(\theta_i)}{(2\pi)^2}\,2\Re \left\lbrace X^{(1)*}(\mathbf{u},\mathbf{q})\,W(\mathbf{u}-\mathbf{q})\\
\iint\frac{d^2r}{(2\pi)^2}W(\mathbf{u}-\mathbf{r})\,W(\mathbf{r}-\mathbf{q})\,
\left[2\,X^{(3)}_a(\mathbf{u},\mathbf{q})^2  \\
+ X^{(3)}_b(\mathbf{u},\mathbf{r},\mathbf{q}) \left(X^{(3)}_b\,(\mathbf{u},\mathbf{r},\mathbf{q})+X^{(3)}_b(\mathbf{u},\mathbf{q-u+r},\mathbf{q})\right) \\
+ X^{(3)}_c(\mathbf{u},\mathbf{q},\mathbf{r},\mathbf{q})\,\left(X^{(3)}_c(\mathbf{u},\mathbf{r},\mathbf{u},\mathbf{q})+X^{(3)}_c(\mathbf{u},\mathbf{r},\mathbf{q-u+r},\mathbf{q}) \right)\\
+X^{(3)}_a(\mathbf{u},\mathbf{q})\,\left(2\,X^{(3)}_b(\mathbf{u},\mathbf{r},\mathbf{q})+
X^{(3)}_b(\mathbf{u},\mathbf{u},\mathbf{q}) + X^{(3)}_b(\mathbf{u},\mathbf{q-u+r},\mathbf{q}) \right)\\
+X^{(3)}_a(\mathbf{u},\mathbf{q})\,\left(2\,X^{(3)}_c(\mathbf{u},\mathbf{q},\mathbf{r},\mathbf{q})+
X^{(3)}_c(\mathbf{u},\mathbf{r},\mathbf{u},\mathbf{q}) + X^{(3)}_c(\mathbf{u},\mathbf{r},\mathbf{q-u+r},\mathbf{q})\right)\\
+X^{(3)}_b(\mathbf{u},\mathbf{r},\mathbf{q})\,\left(X^{(3)}_c(\mathbf{u},\mathbf{r},\mathbf{u},\mathbf{q}) +
X^{(3)}_c(\mathbf{u},\mathbf{r},\mathbf{q-u+r},\mathbf{q})\right) \\
+X^{(3)}_c(\mathbf{u},\mathbf{q},\mathbf{r},\mathbf{q})\,\left(X^{(3)}_b(\mathbf{u},\mathbf{u},\mathbf{q})+X^{(3)}_b(\mathbf{u},\mathbf{q-u+r},\mathbf{q}) \right)
\right] \right\rbrace
$$

In [ ]:
def I3_QP_u(rx,ry,ki,ths,phs,thi,phi,e0,e1,e2,d,s,l):
    
    ksx = kx(ki,ths,phs)
    ksy = ky(ki,ths,phs)
    
    kix = kx(ki,thi,phi)
    kiy = ky(ki,thi,phi)
    
    DKx_sr = ksx-rx
    DKy_sr = ksy-ry
    
    DKx_ri = rx-kix
    DKy_ri = ry-kiy
    
    DKX = kix-ksx+rx
    DKY = kiy-ksy+ry
    
    WW = W(ksx-rx,ksy-ry,s,l)*W(rx-kix,ry-kiy,s,l)
    
    
    ## VV
    
    t1_vv = 2*X3_uA(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,0]**2
    t2_vv = X3_uB(ksx,ksy,rx,ry,kix,kiy,ki,ep0,ep1,ep2,d)[0,0]*(X3_uB(ksx,ksy,DKX,DKY,kix,kiy,ki,ep0,ep1,ep2,d)[0,0]+\
                                                                X3_uB(ksx,ksy,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,0])
    t3_vv = X3_uC(ksx,ksy,kix,kiy,rx,ry,kix,kiy,ki,ep0,ep1,ep2,d)[0,0]*(X3_uC(ksx,ksy,rx,ry,DKX,DKY,kix,kiy,ki,ep0,ep1,ep2,d)[0,0]+\
                                                                       X3_uC(ksx,ksy,rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1,ep0,d)[0,0])
    t4_vv = X3_uA(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,0]*(2*X3_uB(ksx,ksy,rx,ry,kix,kiy,ki,ep0,ep1,ep2,d)[0,0]+\
                                                         X3_uB(ksx,ksy,DKX,DKY,kix,kiy,ki,ep0,ep1,ep2,d)[0,0]+\
                                                         X3_uB(ksx,ksy,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,0])
    t5_vv = X3_uA(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,0]*(X3_uC(ksx,ksy,kix,kiy,rx,ry,kix,kiy,ki,ep0,ep1,ep2,d)[0,0]+\
                                                         X3_uC(ksx,ksy,rx,ry,DKX,DKY,kix,kiy,ki,ep0,ep1,ep2,d)[0,0]+\
                                                         X3_uC(ksx,ksy,rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,0])
    t6_vv = X3_uB(ksx,ksy,rx,ry,kix,kiy,ki,ep0,ep1,ep2,d)[0,0]*(X3_uC(ksx,ksy,rx,ry,DKX,DKY,kix,kiy,ki,ep0,ep1,ep2,d)[0,0]+\
                                                               X3_uC(ksx,ksy,rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,0])
    t7_vv = X3_uC(kix,ksy,kix,kiy,rx,ry,kix,kiy,ki,ep0,ep1,ep2,d)[0,0]*(X3_uB(ksx,ksy,DKX,DKY,kix,kiy,ki,ep0,ep1,ep2,d)[0,0]+\
                                                                      X3_uB(ksx,ksy,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,0])
    
    vv = WW*(t1_vv+t2_vv+t3_vv+t4_vv+t5_vv+t6_vv+t7_vv)
    
    
    ## HV
    
    t1_hv = 2*X3_uA(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,1]**2
    t2_hv = X3_uB(ksx,ksy,rx,ry,kix,kiy,ki,ep0,ep1,ep2,d)[0,1]*(X3_uB(ksx,ksy,DKX,DKY,kix,kiy,ki,ep0,ep1,ep2,d)[0,1]+\
                                                                X3_uB(ksx,ksy,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,1])
    t3_hv = X3_uC(ksx,ksy,kix,kiy,rx,ry,kix,kiy,ki,ep0,ep1,ep2,d)[0,1]*(X3_uC(ksx,ksy,rx,ry,DKX,DKY,kix,kiy,ki,ep0,ep1,ep2,d)[0,1]+\
                                                                       X3_uC(ksx,ksy,rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1,ep0,d)[0,1])
    t4_hv = X3_uA(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,1]*(2*X3_uB(ksx,ksy,rx,ry,kix,kiy,ki,ep0,ep1,ep2,d)[0,1]+\
                                                         X3_uB(ksx,ksy,DKX,DKY,kix,kiy,ki,ep0,ep1,ep2,d)[0,1]+\
                                                         X3_uB(ksx,ksy,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,1])
    t5_hv = X3_uA(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,1]*(X3_uC(ksx,ksy,kix,kiy,rx,ry,kix,kiy,ki,ep0,ep1,ep2,d)[0,1]+\
                                                         X3_uC(ksx,ksy,rx,ry,DKX,DKY,kix,kiy,ki,ep0,ep1,ep2,d)[0,1]+\
                                                         X3_uC(ksx,ksy,rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,1])
    t6_hv = X3_uB(ksx,ksy,rx,ry,kix,kiy,ki,ep0,ep1,ep2,d)[0,1]*(X3_uC(ksx,ksy,rx,ry,DKX,DKY,kix,kiy,ki,ep0,ep1,ep2,d)[0,1]+\
                                                               X3_uC(ksx,ksy,rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,1])
    t7_hv = X3_uC(kix,ksy,kix,kiy,rx,ry,kix,kiy,ki,ep0,ep1,ep2,d)[0,1]*(X3_uB(ksx,ksy,DKX,DKY,kix,kiy,ki,ep0,ep1,ep2,d)[0,1]+\
                                                                      X3_uB(ksx,ksy,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[0,1])
    
    hv = WW*(t1_hv+t2_hv+t3_hv+t4_hv+t5_hv+t6_hv+t7_hv)
    
    
    ## VH
    
    t1_vh = 2*X3_uA(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,0]**2
    t2_vh = X3_uB(ksx,ksy,rx,ry,kix,kiy,ki,ep0,ep1,ep2,d)[1,0]*(X3_uB(ksx,ksy,DKX,DKY,kix,kiy,ki,ep0,ep1,ep2,d)[1,0]+\
                                                                X3_uB(ksx,ksy,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,0])
    t3_vh = X3_uC(ksx,ksy,kix,kiy,rx,ry,kix,kiy,ki,ep0,ep1,ep2,d)[1,0]*(X3_uC(ksx,ksy,rx,ry,DKX,DKY,kix,kiy,ki,ep0,ep1,ep2,d)[1,0]+\
                                                                       X3_uC(ksx,ksy,rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1,ep0,d)[1,0])
    t4_vh = X3_uA(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,0]*(2*X3_uB(ksx,ksy,rx,ry,kix,kiy,ki,ep0,ep1,ep2,d)[1,0]+\
                                                         X3_uB(ksx,ksy,DKX,DKY,kix,kiy,ki,ep0,ep1,ep2,d)[1,0]+\
                                                         X3_uB(ksx,ksy,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,0])
    t5_vh = X3_uA(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,0]*(X3_uC(ksx,ksy,kix,kiy,rx,ry,kix,kiy,ki,ep0,ep1,ep2,d)[1,0]+\
                                                         X3_uC(ksx,ksy,rx,ry,DKX,DKY,kix,kiy,ki,ep0,ep1,ep2,d)[1,0]+\
                                                         X3_uC(ksx,ksy,rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,0])
    t6_vh = X3_uB(ksx,ksy,rx,ry,kix,kiy,ki,ep0,ep1,ep2,d)[1,0]*(X3_uC(ksx,ksy,rx,ry,DKX,DKY,kix,kiy,ki,ep0,ep1,ep2,d)[1,0]+\
                                                               X3_uC(ksx,ksy,rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,0])
    t7_vh = X3_uC(kix,ksy,kix,kiy,rx,ry,kix,kiy,ki,ep0,ep1,ep2,d)[1,0]*(X3_uB(ksx,ksy,DKX,DKY,kix,kiy,ki,ep0,ep1,ep2,d)[1,0]+\
                                                                      X3_uB(ksx,ksy,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,0])
    
    vh = WW*(t1_vh+t2_vh+t3_vh+t4_vh+t5_vh+t6_vh+t7_vh)
    
    
    ## HH
    
    t1_hh = 2*X3_uA(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,1]**2
    t2_hh = X3_uB(ksx,ksy,rx,ry,kix,kiy,ki,ep0,ep1,ep2,d)[1,1]*(X3_uB(ksx,ksy,DKX,DKY,kix,kiy,ki,ep0,ep1,ep2,d)[1,1]+\
                                                                X3_uB(ksx,ksy,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,1])
    t3_hh = X3_uC(ksx,ksy,kix,kiy,rx,ry,kix,kiy,ki,ep0,ep1,ep2,d)[1,1]*(X3_uC(ksx,ksy,rx,ry,DKX,DKY,kix,kiy,ki,ep0,ep1,ep2,d)[1,1]+\
                                                                       X3_uC(ksx,ksy,rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1,ep0,d)[1,1])
    t4_hh = X3_uA(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,1]*(2*X3_uB(ksx,ksy,rx,ry,kix,kiy,ki,ep0,ep1,ep2,d)[1,1]+\
                                                         X3_uB(ksx,ksy,DKX,DKY,kix,kiy,ki,ep0,ep1,ep2,d)[1,1]+\
                                                         X3_uB(ksx,ksy,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,1])
    t5_hh = X3_uA(ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,1]*(X3_uC(ksx,ksy,kix,kiy,rx,ry,kix,kiy,ki,ep0,ep1,ep2,d)[1,1]+\
                                                         X3_uC(ksx,ksy,rx,ry,DKX,DKY,kix,kiy,ki,ep0,ep1,ep2,d)[1,1]+\
                                                         X3_uC(ksx,ksy,rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,1])
    t6_hh = X3_uB(ksx,ksy,rx,ry,kix,kiy,ki,ep0,ep1,ep2,d)[1,1]*(X3_uC(ksx,ksy,rx,ry,DKX,DKY,kix,kiy,ki,ep0,ep1,ep2,d)[1,1]+\
                                                               X3_uC(ksx,ksy,rx,ry,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,1])
    t7_hh = X3_uC(kix,ksy,kix,kiy,rx,ry,kix,kiy,ki,ep0,ep1,ep2,d)[1,1]*(X3_uB(ksx,ksy,DKX,DKY,kix,kiy,ki,ep0,ep1,ep2,d)[1,1]+\
                                                                      X3_uB(ksx,ksy,ksx,ksy,kix,kiy,ki,ep0,ep1,ep2,d)[1,1])
    
    hh = WW*(t1_hh+t2_hh+t3_hh+t4_hh+t5_hh+t6_hh+t7_hh)
    
    
    out = np.array([(vv,hv),(vh,vv)])
    
    return out


# <span style= color:red> Layered rough surfaces </span>

### Matrices auxiliares ### 

In [3]:
def epsilon(ep0,ep1):
    
    m11 = 0.5*1/np.sqrt(ep0*ep1)
    m12 = 0
    m21 = 0
    m22 = 0.5
    
    m = np.array([(m11,m12),(m21,m22)])
    
    return m


def ep_Dmenos(qx,qy,ki,ep0,ep1):
    
    l11 = 0.5*(ep1*alpha(ki,qx,qy,ep0)-ep0*alpha(ki,qx,qy,ep1))/np.sqrt(ep0*ep1)
    l22 = 0.5*(alpha(ki,qx,qy,ep0)-alpha(ki,qx,qy,ep1))
    
    m = np.array([(l11,0),(0,l22)])
    
    return m


def S_mas(ux,uy,qx,qy,ki,ep0,ep1,ep2,d):
    
    U = np.sqrt(ux**2+uy**2)
    Q = np.sqrt(qx**2+qy**2)
    
    u_dot_q = (ux*qx+uy*qy)/(U*Q)
    u_X_q = (ux*qy-uy*qx)/(U*Q)
    
    f = (ep1-ep0)/np.sqrt(ep0*ep1)
    
    a = 1/(ep1*alpha(ki,qx,qy,ep0)+ep0*alpha(ki,qx,qy,ep1))
    b = 1/(alpha(ki,qx,qy,ep0)+alpha(ki,qx,qy,ep1))
    
    
    m11 = f*a*(ep1*U*Q*Fmas_v(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)+\
          ep0*alpha(ki,ux,uy,ep1)*alpha(ki,qx,qy,ep1)*Fmenos_v(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)*u_dot_q)
    
    m12 = f*b*np.sqrt(ep0)*ki*alpha(ki,ux,uy,ep1)*Fmas_h(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)*u_X_q
    
    m21 = -f*a*ep0*np.sqrt(ep1)*ki*alpha(ki,qx,qy,ep1)*Fmenos_v(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)*u_X_q
    
    m22 = f*b*np.sqrt(ep0*ep1)*ki**2*Fmas_h(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)*u_dot_q
    
    m = np.array([(m11,m12),(m21,m22)])
    
    return m




def S_menos(ux,uy,qx,qy,ki,ep0,ep1,ep2,d):
    
    U = np.sqrt(ux**2+uy**2)
    Q = np.sqrt(qx**2+qy**2)
    
    u_dot_q = (ux*qx+uy*qy)/(U*Q)
    u_X_q = (ux*qy-uy*qx)/(U*Q)
    
    f = (ep1-ep0)/(alpha(ki,qx,qy,ep0)*np.sqrt(ep0*ep1))
    
    a = 1/(ep1*alpha(ki,qx,qy,ep0)+ep0*alpha(ki,qx,qy,ep1))
    b = 1/(alpha(ki,qx,qy,ep0)+alpha(ki,qx,qy,ep1))
    
    m11 = -f*a*(ep0*alpha(ki,qx,qy,ep1)*U*Q*Fmenos_v(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)+\
            ep1*alpha(ki,ux,uy,ep1)*alpha(ki,qx,qy,ep0)**2*Fmas_v(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)*u_dot_q)
    
    m12 = -f*b*np.sqrt(ep0)*ki*alpha(ki,ux,uy,ep1)*alpha(ki,qx,qy,ep1)*Fmenos_h(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)*u_X_q
    
    m21 = f*a*np.sqrt(ep1**3)*ki*alpha(ki,qx,qy,ep0)**2*Fmas_v(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)*u_X_q
    
    m22 = -f*b*np.sqrt(ep0*ep1)*ki**2*alpha(ki,qx,qy,ep1)*Fmenos_h(qx,qy,qx,qy,ki,ep0,ep1,ep2,d)*u_dot_q
    
    m = np.array([(m11,m12),(m21,m22)])
        
    return m




def M1b_0a(ux,uy,qx,qy,ki,ep0,ep1,b,a):
    
    U = np.sqrt(ux**2+uy**2)
    Q = np.sqrt(qx**2+qy**2)
    
    u_dot_q = (ux*qx+uy*qy)/(U*Q)
    u_X_q = (ux*qy-uy*qx)/(U*Q)
    
    m11 = U*Q + a*b*alpha(ki,ux,uy,ep1)*alpha(ki,qx,qy,ep0)*u_dot_q
    m12 = -b*np.sqrt(ep0)*ki*alpha(ki,ux,uy,ep1)*u_X_q
    m21 = a*np.sqrt(ep1)*ki*alpha(ki,qx,qy,ep0)*u_X_q
    m22 = np.sqrt(ep0*ep1)*ki**2*u_dot_q
    
    m = np.array([(m11,m12),(m21,m22)])
    
    return m

## Coeficientes de orden cero ##

In [26]:
def X00(qx,qy,ki,ep0,ep1,ep2,d):
    
    r01h = X0(qx,qy,ki,ep0,ep1)[1,1]
    r12h = X0(qx,qy,ki,ep1,ep2)[1,1]
    
    r01v = X0(qx,qy,ki,ep0,ep1)[0,0]
    r12v = X0(qx,qy,ki,ep1,ep2)[0,0]
    
    rv = (r01v + r12v*np.exp(1j*2*alpha(ki,qx,qy,ep1)*d))/(1 + r01v*r12v*np.exp(1j*2*alpha(ki,qx,qy,ep1)*d))
    rh = (r01h + r12h*np.exp(1j*2*alpha(ki,qx,qy,ep1)*d))/(1 + r01h*r12h*np.exp(1j*2*alpha(ki,qx,qy,ep1)*d))
    
    m = np.array([(rv,0),(0,rh)])
    
    return m

## Rough surface on the upper side ##

las ecuaciones (43) a (45) ya están calculadas en las funciones:

    (43) ----> S1_QP_u
    (44) ----> I2_QP_u
    (45) ----> I3_QP_u
    
 
## Rough surface on the bottom side ## 
    
las ecuaciones (46) a (48) ya están calculadas en las funciones:

    (46) ----> S1_QP_b
    (47) ----> I2_QP_b
    (48) ----> I3_QP_b

## Coeficientes de orden dos ##

### ecuaciones (69) y (70)

In [4]:
def X11_21(ux,uy,rx,ry,qx,qy,ki,ep0,ep1,ep2,d):
    
    f_ur = np.exp(1j*d*(alpha(ki,ux,uy,ep1)+alpha(ki,rx,ry,ep1)))
    
    m1 = f_ur*matrix_prod(T10p_U0p(ux,uy,qx,qy,ki,ep0,ep1,ep2,d),X1(ux,uy,rx,ry,ki,ep1,ep2))
    m2 = -matrix_prod(ep_Dmenos(rx,ry,ki,ep0,ep1),X1_u(rx,ry,qx,qy,ki,ep0,ep1,ep2,d)) +\
        1j*S_mas(rx,ry,qx,qy,ki,ep0,ep1,ep2,d)
    
    m = matrix_prod(m1,m2)
    
    return m



def X11_12(ux,uy,rx,ry,qx,qy,ki,ep0,ep1,ep2,d):
    
    m = 1j*matrix_prod(Pmas(ux,uy,rx,ry,ki,ep0,ep1,ep2,d),X1_b(rx,ry,qx,qy,ki,ep0,ep1,ep2,d))
    
    return m*0

In [34]:
def I11_11(rx,ry,ux,uy,qx,qy,ki,ep0,ep1,ep2,d,s1,l1,s2,l2):
    
    X_a = W(ux-rx,uy-ry,s1,l1)*W(rx-qx,ry-qy,s2,l2)*X11_12(ux,uy,rx,ry,qx,qy,ki,ep0,ep1,ep2,d)
    X_b = X11_12(ux,uy,rx,ry,qx,qy,ki,ep0,ep1,ep2,d) + X11_21(ux,uy,ux+qx-rx,uy+qy-ry,qx,qy,ki,ep0,ep1,ep2,d)
    
    X_c = W(ux-rx,uy-ry,s2,l2)*W(rx-qx,ry-qy,s1,l1)*X11_21(ux,uy,rx,ry,qx,qy,ki,ep0,ep1,ep2,d)
    X_d = X11_21(ux,uy,rx,ry,qx,qy,ki,ep0,ep1,ep2,d) + X11_12(ux,uy,ux+qx-rx,uy+qy-ry,qx,qy,ki,ep0,ep1,ep2,d)
    
    m00 = X_a[0,0]*np.conj(X_b[0,0]) + X_c[0,0]*np.conj(X_d[0,0])
    m01 = X_a[0,1]*np.conj(X_b[0,1]) + X_c[0,1]*np.conj(X_d[0,1])
    m10 = X_a[1,0]*np.conj(X_b[1,0]) + X_c[1,0]*np.conj(X_d[1,0])
    m11 = X_a[1,1]*np.conj(X_b[1,1]) + X_c[1,1]*np.conj(X_d[1,1])
    
    m = np.array([(m00,m01),(m10,m11)])
    
    return m

## Coeficiente de orden dos ##

### ecuaciones (74), (75) y (76)

In [35]:
def X12_122(ux,uy,rx,ry,tx,ty,qx,qy,ki,ep0,ep1,ep2,d):
    
    m = 1j*matrix_prod(Pmas(ux,uy,rx,ry,ki,ep0,ep1,ep2,d),X2_b(rx,ry,tx,ty,qx,qy,ki,ep0,ep1,ep2,d))
    
    return m


def X12_212(ux,uy,rx,ry,tx,ty,qx,qy,ki,ep0,ep1,ep2,d):
    
    f_ur = np.exp(1j*d*(alpha(ki,ux,uy,ep1)+alpha(ki,rx,ry,ep1)))
    
    m1 = f_ur*matrix_prod(T10p_U0p(ux,uy,qx,qy,ki,ep0,ep1,ep2,d),X1(ux,uy,rx,ry,ki,ep1,ep2))
    m2 = - matrix_prod(ep_Dmenos(rx,ry,ki,ep0,ep1),X11_12(rx,ry,tx,ty,qx,qy,ki,ep0,ep1,ep2,d))
    m3 = 0.5*1j*(ep1-ep0)/np.sqrt(ep0*ep1)*matrix_prod(M1b_0a(rx,ry,tx,ty,ki,ep0,ep1,-1,1),X1_b(tx,ty,qx,qy,ki,ep0,ep1,ep2,d))
    m4 = m2+m3
    
    m = matrix_prod(m1,m4)
    
    return m


def X12_221(ux,uy,rx,ry,tx,ty,qx,qy,ki,ep0,ep1,ep2,d):
    
    f_ur = np.exp(1j*d*(alpha(ki,ux,uy,ep1)+alpha(ki,rx,ry,ep1)))
    f_ut = np.exp(1j*d*(alpha(ki,ux,uy,ep1)+alpha(ki,tx,ty,ep1)))
    
    m1 = -f_ur*matrix_prod(X1(ux,uy,rx,ry,ki,ep1,ep2),ep_Dmenos(rx,ry,ki,ep0,ep1))
    m2 = matrix_prod(m1,X11_21(rx,ry,tx,ty,qx,qy,ki,ep0,ep1,ep2,d))
    
    m3 = -matrix_prod(ep_Dmenos(tx,ty,ki,ep0,ep1),X1_u(tx,ty,qx,qy,ki,ep0,ep1,ep2,d))+1j*S_mas(tx,ty,qx,qy,ki,ep0,ep1,ep2,d)
    m4 = f_ut*matrix_prod(X2(ux,uy,rx,ry,tx,ty,ki,ep1,ep2),m3)
    
    m = matrix_prod(T10_p_U0_p(ux,uy,qx,qy,ki,ep0,ep1,ep2,d),m2+m4)
    
    return m

In [36]:
def I12_10(rx,ry,ux,uy,qx,qy,ki,d,ep0,ep1,ep2,s1,l1,s2,l2):
    
    X_a = W(ux-rx,uy-ry,s2,l2)*(X12_221(ux,uy,rx,ry,ux,uy,qx,qy,ki,ep0,ep1,ep2,d) +\
                                X12_212(ux,uy,rx,ry,qx-ux+rx,qy-uy+ry,qx,qy,ki,ep0,ep1,ep2,d))
    X_b = W(rx-qx,ry-qy,s2,l2)*X12_122(ux,uy,qx,qy,rx,ry,qx,qy,ki,ep0,ep1,ep2,d)

    X_c = W(ux-qx,uy-qy,s1,l1)*X1_u(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)
    
    m00 = 2*np.real((X_a[0,0] + X_b[0,0])*np.conj(X_c[0,0]))
    m01 = 2*np.real((X_a[0,1] + X_b[0,1])*np.conj(X_c[0,1]))
    m10 = 2*np.real((X_a[1,0] + X_b[1,0])*np.conj(X_c[1,0]))
    m11 = 2*np.real((X_a[1,1] + X_b[1,1])*np.conj(X_c[1,1]))
    
    m = np.array([(m00,m01),(m10,m11)])
    
    return m
    

## Coeficiente de orden dos ##

### ecuaciones (71), (72) y (73) ###

In [37]:
def X21_112(ux,uy,rx,ry,tx,ty,qx,qy,ki,ep0,ep1,ep2,d):
    
    m1 = 1j*matrix_prod(Pmas(ux,uy,rx,ry,ki,ep0,ep1,ep2,d),X11_12(rx,ry,tx,ty,qx,qy,ki,ep0,ep1,ep2,d))
    m2 = 0.5*(alpha(ki,ux,uy,ep1)*Pmenos(ux,uy,tx,ty,ki,ep0,ep1,ep2,d)-alpha(ki,tx,ty,ep0)*Pmas(ux,uy,tx,ty,ki,ep0,ep1,ep2,d))
    m3 = matrix_prod(m2,X1_b(tx,ty,qx,qy,ki,ep0,ep1,ep2,d))
    
    m = m1 + m3
    
    return m


def X21_121(ux,uy,rx,ry,tx,ty,qx,qy,ki,ep0,ep1,ep2,d):
    
    m = 1j*matrix_prod(Pmas(ux,uy,rx,ry,ki,ep0,ep1,ep2,d),X11_21(rx,ry,tx,ty,qx,qy,ki,ep0,ep1,ep2,d))
    
    return m


def X21_211(ux,uy,rx,ry,tx,ty,qx,qy,ki,ep0,ep1,ep2,d):
    
    f_ur = np.exp(1j*d*(alpha(ki,ux,uy,ep1)+alpha(ki,rx,ry,ep1)))
    
    m1 = f_ur*matrix_prod(T10_p_U0_p(ux,uy,qx,qy,ki,ep0,ep1,ep2,d),X1(ux,uy,rx,ry,ki,ep1,ep2))
    
    m2 = -matrix_prod(ep_Dmenos(rx,ry,ki,ep0,ep1),X2_u(rx,ry,tx,ty,qx,qy,ki,ep0,ep1,ep2,d))
    m3 = 0.5*1j*(ep1-ep0)/np.sqrt(ep0*ep1)*matrix_prod(M1b_0a(rx,ry,tx,ty,ki,ep0,ep1,-1,1),X1_u(tx,ty,qx,qy,ki,ep0,ep1,ep2,d))
    m4 = -0.5*alpha(ki,rx,ry,ep1)*S_mas(rx,ry,qx,qy,ki,ep0,ep1,ep2,d)-0.5*alpha(ki,qx,qy,ep0)*S_menos(rx,ry,qx,qy,ki,ep0,ep1,ep2,d)
    m5 = m2+m3+m4
    
    m = matrix_prod(m1,m5)
    
    return m


In [38]:
def I21_01(rx,ry,ux,uy,qx,qy,ki,ep0,ep1,ep2,d,s1,l1,s2,l2):
    
    X_a = W(ux-rx,uy-ry,s1,l1)*(X21_112(ux,uy,rx,ry,ux,uy,qx,qy,ki,ep0,ep1,ep2,d)+\
                                X21_121(ux,uy,rx,ry,qx-ux+rx,qy-uy+ry,qx,qy,ki,ep0,ep1,ep2,d))
    
    X_b = W(rx-qx,ry-qy,s1,l1)*X21_211(ux,uy,qx,qy,rx,ry,qx,qy,ki,ep0,ep1,ep2,d)

    X_c = W(ux-qx,uy-qy,s2,l2)*X1_b(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)
    
    m00 = 2*np.real((X_a[0,0] + X_b[0,0])*np.conj(X_c[0,0]))
    m01 = 2*np.real((X_a[0,1] + X_b[0,1])*np.conj(X_c[0,1]))
    m10 = 2*np.real((X_a[1,0] + X_b[1,0])*np.conj(X_c[1,0]))
    m11 = 2*np.real((X_a[1,1] + X_b[1,1])*np.conj(X_c[1,1]))
    
    m = np.array([(m00,m01),(m10,m11)])
    
    return m

# <span style= color:blue> Berginc 2007 </span> 

## potencias del espectro de rugosidad (gaussiano)


In [ ]:
def W_n(n,kx,ky,s,l):
    
    return s**(2*n)*l**2/(4*np.pi*n)*np.exp(-(kx**2+ky**2)*l**2/(4*n))



# defino la funcion factorial
def factorial(n):
    return prod(range(1,n+1))

## Primer Grupo de Términos ##

$ \langle \,\, R^{(1,0)}\, R^{(1,0)*}\,\, \rangle \,(\mathbf{p}, \mathbf{p}_0) $

### Defino coeficientes $\Sigma_u$ y  $\Sigma_b$

In [ ]:
# Rough surface at the upper side

def sigma0_u(rx,ry,ux,uy,qx,qy,ki,ep0,ep1,ep2,d):
    
    suma = X2_uA(ux,uy,qx,qy,ki,ep0,ep1,ep2,d) + X2_uB(ux,uy,ux-rx,uy-ry,qx,qy,ki,ep0,ep1,ep2,d) +\
           X2_uA(ux,uy,qx,qy,ki,ep0,ep1,ep2,d) + X2_uB(ux,uy,qx+rx,qy+ry,qx,qy,ki,ep0,ep1,ep2,d) +\
           1j*(alpha(ki,qx,qy,ep0)+alpha(ki,ux,uy,ep0))*X1_u(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)
    
    return suma


def sigma_u(rx,ry,ux,uy,qx,qy,ki,ep0,ep1,ep2,d):
    
    t1 = sigma0_u(rx,ry,ux,uy,qx,qy,ki,ep0,ep1,ep2,d)
    t2 = sigma0_u(-rx,-ry,-qx,-qy,-ux,-uy,ki,ep0,ep1,ep2,d)
    
    m00 = 0.5*(t1[0,0]+t2[0,0])
    m01 = 0.5*(t1[0,1]-t2[1,0])
    m10 = 0.5*(t1[1,0]-t2[0,1])
    m11 = 0.5*(t1[1,1]+t2[1,1])
    
    m = np.array([(m00,m01),(m10,m11)])
    
    return m


# Rough surface at the bottom side

def sigma0_b(rx,ry,ux,uy,qx,qy,ki,ep0,ep1,ep2,d):
    
    suma = X2_b(ux,uy,ux-rx,uy-ry,qx,qy,ki,ep0,ep1,ep2,d) +\
           X2_b(ux,uy,qx+rx,qy+ry,qx,qy,ki,ep0,ep1,ep2,d) +\
           1j*(alpha(ki,qx,qy,ep0)+alpha(ki,ux,uy,ep0))*X1_b(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)
    
    return suma


def sigma_b(rx,ry,ux,uy,qx,qy,ki,ep0,ep1,ep2,d):
    
    t1 = sigma0_b(rx,ry,ux,uy,qx,qy,ki,ep0,ep1,ep2,d)
    t2 = sigma0_b(-rx,-ry,-qx,-qy,-ux,-uy,ki,ep0,ep1,ep2,d)
    
    m00 = 0.5*(t1[0,0]+t2[0,0])
    m01 = 0.5*(t1[0,1]-t2[1,0])
    m10 = 0.5*(t1[1,0]-t2[0,1])
    m11 = 0.5*(t1[1,1]+t2[1,1])
    
    m = np.array([(m00,m01),(m10,m11)])
    
    return m

In [ ]:
# defino prefactor

def f_10(ki,ths,phs,thi,phi,ep0,s):
    
    qx = ki*np.sin(thi)*np.cos(phi)
    qy = ki*np.sin(thi)*np.sin(phi)
    ux = ki*np.sin(ths)*np.cos(phs)
    uy = ki*np.sin(ths)*np.sin(phs)
    
    f = -(alpha(ki,ux,uy,ep0)*alpha(ki,qx,qy,ep0))/(alpha(ki,ux,uy,ep0)+alpha(ki,qx,qy,ep0))**2
    g = np.exp(-0.5*s**2*(alpha(ki,ux,uy,ep0)+alpha(ki,qx,qy,ep0))**2)
    
    return f*g

In [ ]:
def SumaO1(ki,ths,phs,th,ph,s1,l1,s2,l2,d,nn):
    
    qx = ki*np.sin(thi)*np.cos(phi)
    qy = ki*np.sin(thi)*np.sin(phi)
    ux = ki*np.sin(ths)*np.cos(phs)
    uy = ki*np.sin(ths)*np.sin(phs)
    
    alpha_s = alpha(ki,ux,uy,ep0)+alpha(ki,qx,qy,ep0)
    
    aux = 0
    
    for t in range(nn): 
        j = t+1
        wj = (alpha_s)**(2*j)/factorial(j)*\
            (f_10(ki,ths,phs,thi,phi,ep0,s1)*np.abs(X1_u(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)[0,0])**2*W_n(j,ux-qx,uy-qy,s1,l1)+\
             f_10(ki,ths,phs,thi,phi,ep0,s2)*np.abs(X1_b(ux,uy,qx,qy,ki,ep0,ep1,ep2,d)[0,0])**2*W_n(j,ux-qx,uy-qy,s2,l2))
        
        aux = aux + wj
    
    return aux